# 1. Importing and installing relevant packages for our project

**Let's install some packages**

In [ ]:
pip install pyradiomics

In [ ]:
pip install -U segmentation-models

**Let's import some packages**

In [ ]:
import os 
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import random
from tqdm import tqdm
import numpy as np
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Dropout, Conv2D, MaxPooling2D, concatenate, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation, UpSampling2D 
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import models 
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import History 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.initializers import RandomNormal
import seaborn 
import seaborn as sns
from tensorflow.keras.optimizers import RMSprop
from sklearn.feature_selection import RFECV
from pathlib import Path
import ast
import time
from IPython import display
from IPython.display import clear_output
%env SM_FRAMEWORK=tf.keras

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')

In [ ]:
from __future__ import print_function
import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase, glcm, glrlm, glszm, imageoperations, shape, getImageTypes, getFeatureClasses, getParameterValidationFiles

# 2. Defining variables

In [ ]:
SIZE = 256
CHANNELS = 1

IMG_HEIGHT, IMG_WIDTH = 256,256
SEED=1
NUM_SAMPLES = 4000
BATCH_SIZE = 1
EPOCHS = 100

In [ ]:
SIZE = 128
CHANNELS = 1

IMG_HEIGHT, IMG_WIDTH = 128,128
SEED=1
NUM_SAMPLES = 4000
BATCH_SIZE = 1
EPOCHS = 100

# 3. Importing datasets path

**DATASET: *XRAY BONE SHADOW SUPRESSION***

In [ ]:
supression_images_dir = os.path.join("../input/xray-bone-shadow-supression/augmented/augmented/source/")
supression_targets_dir = os.path.join("../input/xray-bone-shadow-supression/augmented/augmented/target/")
supression_images_list = os.listdir(supression_images_dir)
supression_target_list = os.listdir(supression_targets_dir)

**DATASET: *RANZCR CLIP LUNG CONTOURS***

PhilippSchwarz explained how to import this dataset in this [notebook](https://www.kaggle.com/philippschwarz/ranzcr-lung-mask-model-not-pretrained).

In [ ]:
TRAIN_PATH = '../input/ranzcr-clip-cxr-dataset/kaggle_images/'
IMAGE_LIB = TRAIN_PATH
ctr = pd.read_csv('../input/ranzcr-clip-lung-contours/RANZCR_CLiP_lung_contours.csv')

**DATASET: *CHEST XRAY MASKS AND LABELS***

In [ ]:
image_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png")
mask_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/","masks/")
images = os.listdir(image_path)
mask = os.listdir(mask_path)

**DATASET: *CHEST X-RAY IMAGES WITH PNEUMOTHORAX MASKS***

In [ ]:
DATADIR_IMAGES = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_images/'
DF_TEST = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv')
DF_TRAIN = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv')

**DATASET: *NIH Chest X-ray Dataset Sample***

In [ ]:
DF_LUNG_DISEASES=pd.read_csv("../input/data/Data_Entry_2017.csv")
LUNG_DISEASES_DIR = os.path.join('../input/data')

In [ ]:
import zipfile
with zipfile.ZipFile("../input/diseases-images//DISEASES.zip","r") as zip_ref:
    zip_ref.extractall("images")

In [ ]:
with zipfile.ZipFile("../input/diseases-attention//DISEASES_ATTENTION.zip","r") as zip_ref:
    zip_ref.extractall("attention_images")

# 4. Catching images

**DATASET: *XRAY BONE SHADOW SUPRESSION***

In [ ]:
supression_images = list()
supression_targets = list()
j = 0
for i in tqdm(supression_target_list):
    if j < NUM_SAMPLES:
        path_images = supression_images_dir + str(i)
        path_targets = supression_targets_dir + str(i)
        image = cv2.imread(path_images,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        image = cv2.bitwise_not(image)
        image = cv2.equalizeHist(image)
        targets = cv2.imread(path_targets,cv2.IMREAD_GRAYSCALE) 
        targets = cv2.resize(targets, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        targets = cv2.bitwise_not(targets)
        targets = cv2.equalizeHist(targets)
        supression_images.append(image)
        supression_targets.append(targets)
    j += 1

In [ ]:
supression_images[0]

In [ ]:
supression_images

In [ ]:
number = 0
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].imshow(supression_images[number], cmap='gray')
ax[1].imshow(supression_targets[number], cmap='gray')
plt.show()

Autoencoder - Supression

In [ ]:
SUPRESSION_IMAGES = (np.array(supression_images).reshape((len(supression_images), SIZE, SIZE, 1))).astype('float32') / 255
SUPRESSION_TARGETS = (np.array(supression_targets).reshape((len(supression_targets), SIZE, SIZE, 1))).astype('float32') / 255
X_train, X_test, y_train, y_test = train_test_split(SUPRESSION_IMAGES, SUPRESSION_TARGETS, test_size=0.30, random_state=12)

**DATASET: *RANZCR CLIP LUNG CONTOURS***

Helper Function (load_mask) to read masks proposed by radda in this [notebook](https://www.kaggle.com/raddar/simple-lung-contour-visualization).

In [ ]:
def load_mask(StudyInstanceUID):
    img = cv2.imread(IMAGE_LIB+StudyInstanceUID+'.jpg',-1)
    ctr_left = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'left_lung_contour'].values[0])
    ctr_right = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'right_lung_contour'].values[0])
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_left]]), 0, (255), -1)
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_right]]), 0, (255), -1)
    img = np.where(img>=255, 1.0, 0.0)
    return img

all_images = os.listdir(TRAIN_PATH)[:NUM_SAMPLES]
all_images = [Path(e).stem for e in all_images]

x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(tqdm(all_images)):
    im = cv2.imread(IMAGE_LIB + name +'.jpg', cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    im = cv2.equalizeHist(im)
    x_data[i] = im

In [ ]:
number = 0
fig, ax = plt.subplots(1,1, figsize = (8,4))
ax.imshow(x_data[number], cmap='gray')
plt.show()

In [ ]:
SEGMENTATION_IMAGES = (np.array(x_data).reshape((len(x_data), SIZE, SIZE, 1)).astype('float32')) / 255 

In [ ]:
import gc
del im
del x_data
gc.collect()

In [ ]:
def load_mask(StudyInstanceUID):
    img = cv2.imread(IMAGE_LIB+StudyInstanceUID+'.jpg',-1)
    ctr_left = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'left_lung_contour'].values[0])
    ctr_right = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'right_lung_contour'].values[0])
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_left]]), 0, (255), -1)
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_right]]), 0, (255), -1)
    img = np.where(img>=255, 1.0, 0.0)
    return img

all_images = os.listdir(TRAIN_PATH)[:NUM_SAMPLES]
all_images = [Path(e).stem for e in all_images]

x_data = list()
x_data_otsu = list()
x_data_canny = list()
for number in tqdm(range(0,len(SEGMENTATION_IMAGES))):
    im = loaded_model_autoencoder.predict(SEGMENTATION_IMAGES[[number]].astype(np.float32))
    im = np.squeeze(im.astype(np.float32))
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', im, cmap='gray')
    im = cv2.imread('./predicted_supression.jpg',1)
    im = im[:,:,1]
    im = cv2.equalizeHist(im)
    x_data.append(im)
    im = cv2.GaussianBlur(im,(5,5),0)
    im_ = cv2.Canny(im,56,56)
    _,im = cv2.threshold(im,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    im = cv2.bitwise_not(im)
    x_data_otsu.append(im)
    x_data_canny.append(im_) 
    
y_data = list()
for i, name in enumerate(tqdm(all_images)):  
    im = load_mask(name)
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
    y_data.append(im)

In [ ]:
number = 0
fig, ax = plt.subplots(1,4, figsize = (8,4))
ax[0].imshow(x_data[number], cmap='gray')
ax[1].imshow(x_data_otsu[number], cmap='gray')
ax[2].imshow(x_data_canny[number], cmap='gray')
ax[3].imshow(y_data[number], cmap='gray')
plt.show()

GAN - Input

In [ ]:
import gc
del im
del im_
gc.collect()

SEGMENTATION_IMAGES_GAN = (np.array(x_data).reshape((len(x_data), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_IMAGES_OTSU_GAN = (np.array(x_data_otsu).reshape((len(x_data_otsu), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_IMAGES_CANNY_GAN = (np.array(x_data_canny).reshape((len(x_data_canny), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_TARGETS_GAN = (np.array(y_data).reshape((len(y_data), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5

In [ ]:
import gc
del x_data
del x_data_otsu
del x_data_canny
del y_data
del SEGMENTATION_IMAGES
#del SEGMENTATION_IMAGES_OTSU
#del SEGMENTATION_IMAGES_CANNY
#del SEGMENTATION_TARGETS
gc.collect()

In [ ]:
train_dataset_X = tf.data.Dataset.from_tensor_slices(SEGMENTATION_IMAGES_GAN).batch(BATCH_SIZE)
train_dataset_otsu_X = tf.data.Dataset.from_tensor_slices(SEGMENTATION_IMAGES_OTSU_GAN).batch(BATCH_SIZE)
train_dataset_canny_X = tf.data.Dataset.from_tensor_slices(SEGMENTATION_IMAGES_CANNY_GAN).batch(BATCH_SIZE)
train_dataset_y = tf.data.Dataset.from_tensor_slices(SEGMENTATION_TARGETS_GAN).batch(BATCH_SIZE)

Autoencoder - Input

In [ ]:
SEGMENTATION_IMAGES = (np.array(x_data).reshape((len(x_data), SIZE, SIZE, 1)).astype('float32')) / 255
SEGMENTATION_TARGETS = (np.array(y_data).reshape((len(y_data), SIZE, SIZE, 1)).astype('float32')) / 255

In [ ]:
X_train_R = list()

for img in SEGMENTATION_IMAGES:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_train_R.append(img_R)
    
X_train_R = np.array(X_train_R)

In [ ]:
import gc
#del im
#del im_
#del x_data
#del x_data_otsu
#del x_data_canny
#del y_data
del SEGMENTATION_IMAGES_GAN
del SEGMENTATION_IMAGES_OTSU_GAN
del SEGMENTATION_IMAGES_CANNY_GAN
del SEGMENTATION_TARGETS_GAN
gc.collect()

**DATASET: *CHEST XRAY MASKS AND LABELS***

Methodology to read images proposed by sumit pandey in this [notebook](https://www.kaggle.com/sumitai/lung-segmentation-from-chest-x-ray-dataset).

In [ ]:
mask = [fName.split(".png")[0] for fName in mask]
image_file_name = [fName.split("_mask")[0] for fName in mask]
check = [i for i in mask if "mask" in i]

testing_files = set(os.listdir(image_path)) & set(os.listdir(mask_path))
training_files = check

ima = list()
j = 0
for i in tqdm(testing_files):
    im = cv2.imread(os.path.join(image_path,i),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    ima.append(im)
    
for i in tqdm(training_files): 
    im = cv2.imread(os.path.join(image_path,i.split("_mask")[0]+".png"),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    ima.append(im)

In [ ]:
number = 0
fig, ax = plt.subplots(1,1, figsize = (8,4))
ax.imshow(ima[number], cmap='gray')
plt.show()

Autoencoder - Supression

In [ ]:
SEGMENTATION_IMAGES_2 = (np.array(ima).reshape((len(ima), SIZE, SIZE, 1)).astype('float32'))/255

In [ ]:
import gc
del im
del ima
gc.collect()

In [ ]:
x_data = list()
x_data_otsu = list()
x_data_canny = list()
for number in tqdm(range(0,len(SEGMENTATION_IMAGES_2))):
    im = loaded_model_autoencoder.predict(SEGMENTATION_IMAGES_2[[number]].astype(np.float32))
    im = np.squeeze(im.astype(np.float32))
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', im, cmap='gray')
    im = cv2.imread('./predicted_supression.jpg',1)
    im = im[:,:,1]
    im = cv2.equalizeHist(im)
    x_data.append(im)
    im = cv2.GaussianBlur(im,(5,5),0)
    im_ = cv2.Canny(im,56,56)
    _,im = cv2.threshold(im,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    im = cv2.bitwise_not(im)
    x_data_otsu.append(im)
    x_data_canny.append(im_) 

In [ ]:
mask = [fName.split(".png")[0] for fName in mask]
image_file_name = [fName.split("_mask")[0] for fName in mask]
check = [i for i in mask if "mask" in i]

testing_files = set(os.listdir(image_path)) & set(os.listdir(mask_path))
training_files = check


mas = list()
j = 0
for i in tqdm(testing_files):
    mask = cv2.imread(os.path.join(mask_path,i),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    _,mask = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mas.append(mask)
    
for i in tqdm(training_files): 
    mask = cv2.imread(os.path.join(mask_path,i+".png"),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    _,mask = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mas.append(mask)

In [ ]:
number = 0
fig, ax = plt.subplots(1,4, figsize = (16,8))
ax[0].imshow(x_data[number], cmap='gray')
ax[1].imshow(x_data_otsu[number], cmap='gray')
ax[2].imshow(x_data_canny[number], cmap='gray')
ax[3].imshow(mas[number], cmap='gray')
plt.show()

GAN - Input

In [ ]:
SEGMENTATION_IMAGES_GAN_3 = (np.array(x_data).reshape((len(x_data), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_IMAGES_OTSU_GAN_3 = (np.array(x_data_otsu).reshape((len(x_data_otsu), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_IMAGES_CANNY_GAN_3 = (np.array(x_data_canny).reshape((len(x_data_canny), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5
SEGMENTATION_TARGETS_GAN_3 = (np.array(mas).reshape((len(mas), SIZE, SIZE, 1)).astype('float32') - 127.5) / 127.5

Autoencoder - Input

In [ ]:
SEGMENTATION_IMAGES_3 = (np.array(x_data).reshape((len(x_data), SIZE, SIZE, 1)).astype('float32')) / 255
SEGMENTATION_TARGETS_3 = (np.array(mas).reshape((len(mas), SIZE, SIZE, 1)).astype('float32')) / 255

In [ ]:
X_test_R = list()

for img in SEGMENTATION_IMAGES_3:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_test_R.append(img_R)
    
X_test_R = np.array(X_test_R)

In [ ]:
import gc
del im
del im_
del mas
del mask
del x_data
del x_data_otsu
del x_data_canny
del y_data
#del SEGMENTATION_IMAGES_2
#del SEGMENTATION_IMAGES_OTSU_2
#del SEGMENTATION_IMAGES_CANNY_2
#del SEGMENTATION_TARGETS_2
gc.collect()

**DATASET: *CHEST X-RAY IMAGES WITH PNEUMOTHORAX MASKS***

In [ ]:
LEN_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0].shape[0]
LEN_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1].shape[0]
LEN_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0].shape[0]
LEN_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1].shape[0]

IMAGE_TRAIN_PNEUMO = list()
IMAGE_TRAIN_NO_PNEUMO = list()
IMAGE_TEST_PNEUMO = list()
IMAGE_TEST_NO_PNEUMO = list()

CLASS_IMAGE_TRAIN_PNEUMO = list()
CLASS_IMAGE_TRAIN_NO_PNEUMO = list()
CLASS_IMAGE_TEST_PNEUMO = list()
CLASS_IMAGE_TEST_NO_PNEUMO = list()

NAME_TRAIN_PNEUMO = list()
NAME_TRAIN_NO_PNEUMO = list()
NAME_TEST_PNEUMO = list()
NAME_TEST_NO_PNEUMO = list()

for i in tqdm(range(0,int(1500))):
    DF_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1]
    IMAGE_NAME = DF_TRAIN_PNEUMO.iloc[i,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_PNEUMO.append(image)
    CLASS_IMAGE_TRAIN_PNEUMO.append(1)
    NAME_TRAIN_PNEUMO.append(IMAGE_NAME)

for j in tqdm(range(0,int(1500))):
    DF_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0]
    IMAGE_NAME = DF_TRAIN_NO_PNEUMO.iloc[j,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_NO_PNEUMO.append(image)
    CLASS_IMAGE_TRAIN_NO_PNEUMO.append(0)
    NAME_TRAIN_NO_PNEUMO.append(IMAGE_NAME)
    
for k in tqdm(range(0,int(250))):
    DF_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1]
    IMAGE_NAME = DF_TEST_PNEUMO.iloc[k,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TEST_PNEUMO.append(image)
    CLASS_IMAGE_TEST_PNEUMO.append(1)
    NAME_TEST_PNEUMO.append(IMAGE_NAME)
    
for l in tqdm(range(0,int(250))):
    DF_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0]
    IMAGE_NAME = DF_TEST_NO_PNEUMO.iloc[l,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TEST_NO_PNEUMO.append(image)
    CLASS_IMAGE_TEST_NO_PNEUMO.append(0)
    NAME_TEST_NO_PNEUMO.append(IMAGE_NAME)


In [ ]:
number = 0
fig, ax = plt.subplots(1,4, figsize = (8,4))
ax[0].imshow(IMAGE_TRAIN_PNEUMO[number], cmap='gray')
ax[1].imshow(IMAGE_TRAIN_NO_PNEUMO[number], cmap='gray')
ax[2].imshow(IMAGE_TEST_PNEUMO[number], cmap='gray')
ax[3].imshow(IMAGE_TEST_NO_PNEUMO[number], cmap='gray')
plt.show()

In [ ]:
CLASS_IMAGE_TRAIN_PNEUMO = np.array(CLASS_IMAGE_TRAIN_PNEUMO)
CLASS_IMAGE_TRAIN_NO_PNEUMO = np.array(CLASS_IMAGE_TRAIN_NO_PNEUMO)
CLASS_IMAGE_TEST_PNEUMO = np.array(CLASS_IMAGE_TEST_PNEUMO)
CLASS_IMAGE_TEST_NO_PNEUMO = np.array(CLASS_IMAGE_TEST_NO_PNEUMO)

CLASS_TRAIN = np.concatenate((CLASS_IMAGE_TRAIN_PNEUMO,CLASS_IMAGE_TRAIN_NO_PNEUMO))
CLASS_TEST = np.concatenate((CLASS_IMAGE_TEST_PNEUMO,CLASS_IMAGE_TEST_NO_PNEUMO))

In [ ]:
import gc
del image
gc.collect()

Autoencoder - Supression

In [ ]:
IMAGE_TRAIN_PNEUMO_ = (np.array(IMAGE_TRAIN_PNEUMO).reshape((len(IMAGE_TRAIN_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TRAIN_NO_PNEUMO_ = (np.array(IMAGE_TRAIN_NO_PNEUMO).reshape((len(IMAGE_TRAIN_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TEST_PNEUMO_ = (np.array(IMAGE_TEST_PNEUMO).reshape((len(IMAGE_TEST_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TEST_NO_PNEUMO_ = (np.array(IMAGE_TEST_NO_PNEUMO).reshape((len(IMAGE_TEST_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255

In [ ]:
import gc
del IMAGE_TRAIN_PNEUMO
del IMAGE_TRAIN_NO_PNEUMO
del IMAGE_TEST_PNEUMO
del IMAGE_TEST_NO_PNEUMO
gc.collect()

In [ ]:
LEN_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0].shape[0]
LEN_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1].shape[0]
LEN_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0].shape[0]
LEN_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1].shape[0]

IMAGE_TRAIN_PNEUMO = list()
IMAGE_TRAIN_PNEUMO_OTSU = list()
IMAGE_TRAIN_PNEUMO_CANNY = list()
IMAGE_TRAIN_NO_PNEUMO = list()
IMAGE_TRAIN_NO_PNEUMO_OTSU = list()
IMAGE_TRAIN_NO_PNEUMO_CANNY = list()
IMAGE_TEST_PNEUMO = list()
IMAGE_TEST_PNEUMO_OTSU = list()
IMAGE_TEST_PNEUMO_CANNY = list()
IMAGE_TEST_NO_PNEUMO = list()
IMAGE_TEST_NO_PNEUMO_OTSU = list()
IMAGE_TEST_NO_PNEUMO_CANNY = list()

for i in tqdm(range(0,int(1500))):
    image = loaded_model_autoencoder.predict(IMAGE_TRAIN_PNEUMO_[[i]].astype(np.float32))
    image = np.squeeze(image.astype(np.float32))
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', image, cmap='gray')
    image = cv2.imread('./predicted_supression.jpg',1)
    image = image[:,:,1]
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_PNEUMO.append(image)
    image = cv2.GaussianBlur(image,(5,5),0)
    image_ = cv2.Canny(image,56,56)
    _,image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.bitwise_not(image)
    IMAGE_TRAIN_PNEUMO_OTSU.append(image)
    IMAGE_TRAIN_PNEUMO_CANNY.append(image_)

for j in tqdm(range(0,int(1500))):
    image = loaded_model_autoencoder.predict(IMAGE_TRAIN_NO_PNEUMO_[[j]].astype(np.float32))
    image = np.squeeze(image.astype(np.float32))
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', image, cmap='gray')
    image = cv2.imread('./predicted_supression.jpg',1)
    image = image[:,:,1]
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_NO_PNEUMO.append(image)
    image = cv2.GaussianBlur(image,(5,5),0)
    image_ = cv2.Canny(image,56,56)
    _,image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.bitwise_not(image)
    IMAGE_TRAIN_NO_PNEUMO_OTSU.append(image)
    IMAGE_TRAIN_NO_PNEUMO_CANNY.append(image_)
    
for k in tqdm(range(0,int(250))):
    image = loaded_model_autoencoder.predict(IMAGE_TEST_PNEUMO_[[k]].astype(np.float32))
    image = np.squeeze(image.astype(np.float32))
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', image, cmap='gray')
    image = cv2.imread('./predicted_supression.jpg',1)
    image = image[:,:,1]
    image = cv2.equalizeHist(image)
    IMAGE_TEST_PNEUMO.append(image)
    image = cv2.GaussianBlur(image,(5,5),0)
    image_ = cv2.Canny(image,56,56)
    _,image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.bitwise_not(image)
    IMAGE_TEST_PNEUMO_OTSU.append(image)
    IMAGE_TEST_PNEUMO_CANNY.append(image_)

for l in tqdm(range(0,int(250))):
    image = loaded_model_autoencoder.predict(IMAGE_TEST_NO_PNEUMO_[[l]].astype(np.float32))
    image = np.squeeze(image.astype(np.float32))
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('predicted_supression.jpg', image, cmap='gray')
    image = cv2.imread('./predicted_supression.jpg',1)
    image = image[:,:,1]
    image = cv2.equalizeHist(image)
    IMAGE_TEST_NO_PNEUMO.append(image)
    image = cv2.GaussianBlur(image,(5,5),0)
    image_ = cv2.Canny(image,56,56)
    _,image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.bitwise_not(image)
    IMAGE_TEST_NO_PNEUMO_OTSU.append(image)
    IMAGE_TEST_NO_PNEUMO_CANNY.append(image_)


In [ ]:
gc.collect()

In [ ]:
number = 0
fig, ax = plt.subplots(1,3, figsize = (8,4))
ax[0].imshow(IMAGE_TRAIN_PNEUMO[number], cmap='gray')
ax[1].imshow(IMAGE_TRAIN_PNEUMO_OTSU[number], cmap='gray')
ax[2].imshow(IMAGE_TRAIN_PNEUMO_CANNY[number], cmap='gray')
plt.show()

Autoencoder - Input

In [ ]:
IMAGE_TRAIN_PNEUMO__ = (np.array(IMAGE_TRAIN_PNEUMO).reshape((len(IMAGE_TRAIN_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TRAIN_NO_PNEUMO__ = (np.array(IMAGE_TRAIN_NO_PNEUMO).reshape((len(IMAGE_TRAIN_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TEST_PNEUMO__ = (np.array(IMAGE_TEST_PNEUMO).reshape((len(IMAGE_TEST_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255
IMAGE_TEST_NO_PNEUMO__ = (np.array(IMAGE_TEST_NO_PNEUMO).reshape((len(IMAGE_TEST_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32')) / 255

In [ ]:
PNEUMOTHORAX_IMAGES_TRAIN = np.concatenate((IMAGE_TRAIN_PNEUMO__,IMAGE_TRAIN_NO_PNEUMO__))
PNEUMOTHORAX_IMAGES_TEST = np.concatenate((IMAGE_TEST_PNEUMO__,IMAGE_TEST_NO_PNEUMO__))

In [ ]:
os.makedirs('./PNEUMOTHORAX/images_segment_pneumothorax_train_auto')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TRAIN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TRAIN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_pneumothorax_train_auto/'+ str(i) + '_train.png',predicted_rx, cmap = 'gray')
os.makedirs('./PNEUMOTHORAX/images_segment_pneumothorax_test_auto')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TEST))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TEST[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_pneumothorax_test_auto/'+ str(i) + '_test.png',predicted_rx, cmap = 'gray')

In [ ]:
import gc
del image
del IMAGE_TRAIN_PNEUMO__
del IMAGE_TRAIN_NO_PNEUMO__
del IMAGE_TEST_PNEUMO__
del IMAGE_TEST_NO_PNEUMO__
gc.collect()

GAN - Input

In [ ]:
IMAGE_TRAIN_PNEUMOx = (np.array(IMAGE_TRAIN_PNEUMO).reshape((len(IMAGE_TRAIN_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TRAIN_PNEUMO_OTSUx = (np.array(IMAGE_TRAIN_PNEUMO_OTSU).reshape((len(IMAGE_TRAIN_PNEUMO_OTSU), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TRAIN_PNEUMO_CANNYx = (np.array(IMAGE_TRAIN_PNEUMO_CANNY).reshape((len(IMAGE_TRAIN_PNEUMO_CANNY), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5

IMAGE_TRAIN_NO_PNEUMOx = (np.array(IMAGE_TRAIN_NO_PNEUMO).reshape((len(IMAGE_TRAIN_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TRAIN_NO_PNEUMO_OTSUx = (np.array(IMAGE_TRAIN_NO_PNEUMO_OTSU).reshape((len(IMAGE_TRAIN_NO_PNEUMO_OTSU), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TRAIN_NO_PNEUMO_CANNYx = (np.array(IMAGE_TRAIN_NO_PNEUMO_CANNY).reshape((len(IMAGE_TRAIN_NO_PNEUMO_CANNY), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5

IMAGE_TEST_PNEUMOx = (np.array(IMAGE_TEST_PNEUMO).reshape((len(IMAGE_TEST_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TEST_PNEUMO_OTSUx = (np.array(IMAGE_TEST_PNEUMO_OTSU).reshape((len(IMAGE_TEST_PNEUMO_OTSU), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TEST_PNEUMO_CANNYx = (np.array(IMAGE_TEST_PNEUMO_CANNY).reshape((len(IMAGE_TEST_PNEUMO_CANNY), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5

IMAGE_TEST_NO_PNEUMOx = (np.array(IMAGE_TEST_NO_PNEUMO).reshape((len(IMAGE_TEST_NO_PNEUMO), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TEST_NO_PNEUMO_OTSUx = (np.array(IMAGE_TEST_NO_PNEUMO_OTSU).reshape((len(IMAGE_TEST_NO_PNEUMO_OTSU), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5
IMAGE_TEST_NO_PNEUMO_CANNYx = (np.array(IMAGE_TEST_NO_PNEUMO_CANNY).reshape((len(IMAGE_TEST_NO_PNEUMO_CANNY), SIZE, SIZE, CHANNELS)).astype('float32') - 127.5) / 127.5

In [ ]:
import gc
del IMAGE_TRAIN_PNEUMO
del IMAGE_TRAIN_NO_PNEUMO
del IMAGE_TRAIN_PNEUMO_OTSU
del IMAGE_TRAIN_NO_PNEUMO_OTSU
del IMAGE_TRAIN_PNEUMO_CANNY
del IMAGE_TRAIN_NO_PNEUMO_CANNY
del IMAGE_TEST_PNEUMO
del IMAGE_TEST_NO_PNEUMO
del IMAGE_TEST_PNEUMO_OTSU
del IMAGE_TEST_NO_PNEUMO_OTSU
del IMAGE_TEST_PNEUMO_CANNY
del IMAGE_TEST_NO_PNEUMO_CANNY
gc.collect()

In [ ]:
PNEUMOTHORAX_IMAGES_TRAIN_GAN = np.concatenate((IMAGE_TRAIN_PNEUMOx,IMAGE_TRAIN_NO_PNEUMOx))
PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN = np.concatenate((IMAGE_TRAIN_PNEUMO_OTSUx,IMAGE_TRAIN_NO_PNEUMO_OTSUx))
PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN = np.concatenate((IMAGE_TRAIN_PNEUMO_CANNYx,IMAGE_TRAIN_NO_PNEUMO_CANNYx))
PNEUMOTHORAX_IMAGES_TEST_GAN = np.concatenate((IMAGE_TEST_PNEUMOx,IMAGE_TEST_NO_PNEUMOx))
PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN = np.concatenate((IMAGE_TEST_PNEUMO_OTSUx,IMAGE_TEST_NO_PNEUMO_OTSUx))
PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN = np.concatenate((IMAGE_TEST_PNEUMO_CANNYx,IMAGE_TEST_NO_PNEUMO_CANNYx))

In [ ]:
import gc
del IMAGE_TRAIN_PNEUMOx
del IMAGE_TRAIN_NO_PNEUMOx
del IMAGE_TRAIN_PNEUMO_OTSUx
del IMAGE_TRAIN_NO_PNEUMO_OTSUx
del IMAGE_TRAIN_PNEUMO_CANNYx
del IMAGE_TRAIN_NO_PNEUMO_CANNYx
del IMAGE_TEST_PNEUMOx
del IMAGE_TEST_NO_PNEUMOx
del IMAGE_TEST_PNEUMO_OTSUx
del IMAGE_TEST_NO_PNEUMO_OTSUx
del IMAGE_TEST_PNEUMO_CANNYx
del IMAGE_TEST_NO_PNEUMO_CANNYx
gc.collect()

In [ ]:
del _
gc.collect()

In [ ]:
os.makedirs('./PNEUMOTHORAX/images_segment_pneumothorax_train_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TRAIN_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TRAIN_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_pneumothorax_train_GAN/'+ str(i) + '_train.png',predicted_rx, cmap = 'gray')
os.makedirs('./PNEUMOTHORAX/images_otsu_segment_pneumothorax_train_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_otsu_segment_pneumothorax_train_GAN/'+ str(i) + '_train.png',predicted_rx, cmap = 'gray')
os.makedirs('./PNEUMOTHORAX/images_canny_segment_pneumothorax_train_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_canny_segment_pneumothorax_train_GAN/'+ str(i) + '_train.png',predicted_rx, cmap = 'gray')

os.makedirs('./PNEUMOTHORAX/images_segment_pneumothorax_test_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TEST_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TEST_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_pneumothorax_test_GAN/'+ str(i) + '_test.png',predicted_rx, cmap = 'gray')
os.makedirs('./PNEUMOTHORAX/images_segment_otsu_pneumothorax_test_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_otsu_pneumothorax_test_GAN/'+ str(i) + '_test.png',predicted_rx, cmap = 'gray')
os.makedirs('./PNEUMOTHORAX/images_segment_canny_pneumothorax_test_GAN')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/images_segment_canny_pneumothorax_test_GAN/'+ str(i) + '_test.png',predicted_rx, cmap = 'gray')
    

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./PNEUMOTHORAX_MASKS.zip', 'w')
 
for folder, subfolders, files in os.walk('./PNEUMOTHORAX'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
im = cv2.imread('../input/pneumothorax/images_segment_pneumothorax_train_GAN/0_image_train.jpg',0)

In [ ]:
im = (im.reshape((256, 256, 1)).astype('float32') - 127.5) / 127.5

In [ ]:
plt.imshow(im, cmap = 'gray')
plt.show()

**DATASET: *NIH Chest X-ray Dataset***

In [ ]:
DF_LUNG_DISEASES

In [ ]:
DF_LUNG_DISEASES["Finding Labels"].value_counts()

In [ ]:
label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)

In [ ]:
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES.head(10)

In [ ]:
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
DF_LUNG_DISEASES

In [ ]:
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES_INTE.head(40198)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
DF_LUNG_DISEASES

In [ ]:
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')
NAMES_JPG_1

In [ ]:
DF_LUNG_DISEASES["Finding Labels"].value_counts()

In [ ]:
sns.set_theme(style="darkgrid")
plt.figure(figsize=(40,20))
sns.countplot(DF_LUNG_DISEASES["Finding Labels"])

In [ ]:
###
IMAGES = list()
NAMES = list()

for i in tqdm(range(1,10)):
    DIR = LUNG_DISEASES_DIR + "/images_00" + str(i) + "/images/"
    LIST = os.listdir(DIR)
    for j in tqdm(LIST):
        PATH = DIR + j
        IMAGE = cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE)
        IMAGE = cv2.resize(IMAGE, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        IMAGES.append(IMAGE)
        NAMES.append(j)
        clear_output(wait=True)
        
for i in tqdm(range(10,13)):
    DIR = LUNG_DISEASES_DIR + "/images_0" + str(i) + "/images/"
    LIST = os.listdir(DIR)
    for j in tqdm(LIST):
        PATH = DIR + j
        IMAGE = cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE)
        IMAGE = cv2.resize(IMAGE, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        IMAGES.append(IMAGE)
        NAMES.append(j)
        clear_output(wait=True)

In [ ]:
NME_LIST = pd.DataFrame({'Image Index': NAMES})
NAMES_JPG_2 = NME_LIST['Image Index'].str.replace('.png', '.jpg')

In [ ]:
os.makedirs('./DISEASES/images')
for i in tqdm(range(0,len(NAMES))):
    plt.imsave('./DISEASES/images/'+ str(NAMES_JPG_2.iloc[i]),IMAGES[i], cmap = 'gray')

In [ ]:
NAMES_JPG_2.to_csv('./DISEASES/image_names', sep=',', encoding='utf-8')

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./DISEASES.zip', 'w')
 
for folder, subfolders, files in os.walk('./DISEASES'):
 
    for file in files:
        if file.endswith('.jpg'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

**LET'S GO**

In [ ]:
DF_LUNG_DISEASES_input = pd.read_csv("../input/diseases-image-names/image_names.csv")
DF_LUNG_DISEASES_input = DF_LUNG_DISEASES_input.drop(columns = 'Unnamed: 0')
DF_LUNG_DISEASES_input

In [ ]:
###
import gc

DIR_ = './images/DISEASES/images/'

os.makedirs('./Diseases/normal')
os.makedirs('./Diseases/otsu')
os.makedirs('./Diseases/canny')

IMAGE_DISEASE = list()
IMAGE_DISEASE_OTSU = list()
IMAGE_DISEASE_CANNY = list()
batch = 2
for h in range(0+batch,1+batch):
    c = int((len(NAMES_JPG_1)/3)*h) 
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/3)+c))):
        PATH = DIR_ + NAMES_JPG_1.iloc[i]
        IMAGE = cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE)
        IMAGE = (np.asarray(IMAGE).reshape(1,SIZE, SIZE, CHANNELS).astype('float16')) / 255
        IMAGE = loaded_model_autoencoder.predict(IMAGE.astype(np.float16))
        IMAGE = np.squeeze(IMAGE.astype(np.float32))
        IMAGE = cv2.resize(IMAGE, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
        plt.imsave('predicted_supression.png', IMAGE, cmap='gray')
        IMAGE = cv2.imread('./predicted_supression.png',1)
        IMAGE = IMAGE[:,:,1]
        IMAGE = cv2.equalizeHist(IMAGE)
        IMAGE_DISEASE.append(IMAGE)
        IMAGE_ = cv2.Canny(IMAGE,56,56)
        IMAGE= cv2.GaussianBlur(IMAGE,(5,5),0)
        _,IMAGE = cv2.threshold(IMAGE,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        IMAGE = cv2.bitwise_not(IMAGE)
        IMAGE_DISEASE_OTSU.append(IMAGE)
        IMAGE_DISEASE_CANNY.append(IMAGE_)
    
    
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/3)+c))):
        predicted_rx = np.squeeze(IMAGE_DISEASE[i-c].astype(np.float16))
        plt.imsave('./Diseases/normal/'+ str(NAMES_JPG_1.iloc[i]),predicted_rx, cmap = 'gray')
    
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/3)+c))):
        predicted_rx = np.squeeze(IMAGE_DISEASE_OTSU[i-c].astype(np.float16))
        plt.imsave('./Diseases/otsu/'+ str(NAMES_JPG_1.iloc[i]),predicted_rx, cmap = 'gray')
    
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/3)+c))):
        predicted_rx = np.squeeze(IMAGE_DISEASE_CANNY[i-c].astype(np.float16))
        plt.imsave('./Diseases/canny/'+ str(NAMES_JPG_1.iloc[i]),predicted_rx, cmap = 'gray')

    del IMAGE
    del IMAGE_DISEASE
    del IMAGE_DISEASE_OTSU
    del IMAGE_DISEASE_CANNY 
    del predicted_rx
    gc.collect()

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./DISEASES_2.zip', 'w')
 
for folder, subfolders, files in os.walk('./Diseases'):
 
    for file in files:
        if file.endswith('.jpg'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

Read the images

In [ ]:
IMAGE_DISEASE_1275 = list()
IMAGE_DISEASE_2550 = list()
IMAGE_DISEASE_OTSU = list()
IMAGE_DISEASE_CANNY = list()

batch = 15

DIR_input = '../input/diseases-eq-images-otsu-canny/Diseases/normal/'
for h in range(0+batch,1+batch):
    c = int((len(NAMES_JPG_1)/16)*h) 
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/16)+c))):
        PATH = DIR_input + NAMES_JPG_1.iloc[i]
        IMAGE_DISEASE_1275.append((cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE).astype('float16')- 127.5) / 127.5)
        IMAGE_DISEASE_2550.append((cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE).astype('float16')) / 255.)    

DIR_input = '../input/diseases-eq-images-otsu-canny/Diseases/otsu/'
for h in range(0+batch,1+batch):
    c = int((len(NAMES_JPG_1)/16)*h) 
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/16)+c))):
        PATH = DIR_input + NAMES_JPG_1.iloc[i]
        IMAGE_DISEASE_OTSU.append((cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE).astype('float16')- 127.5) / 127.5)       

DIR_input = '../input/diseases-eq-images-otsu-canny/Diseases/canny/'
for h in range(0+batch,1+batch):
    c = int((len(NAMES_JPG_1)/16)*h) 
    for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/16)+c))):
        PATH = DIR_input + NAMES_JPG_1.iloc[i]
        IMAGE_DISEASE_CANNY.append((cv2.imread(os.path.join(PATH),cv2.IMREAD_GRAYSCALE).astype('float16')- 127.5) / 127.5)   
        
import gc
gc.collect()

from keras.initializers import RandomNormal
from keras.metrics import MeanIoU
import heapq

class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)

def processing(prediction_,list_prediction):   
    for predicted_rx in tqdm(prediction_):
        img = np.squeeze(predicted_rx.astype(np.float16))
        plt.imsave('predicted_000.jpg', img, cmap='gray')
        img = cv2.imread('./predicted_000.jpg',1)
        img = img[:,:,0]
        min_ = img.min()
        max_ = img.max()
        ret1, img = cv2.threshold(img, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        contours,hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

        len_contours = list()
        for i in range(0,len(contours)):
            len_contours.append(len(contours[i]))

        #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  
        try:
            data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
            two_largest, vals = zip(*data)
        except:
            pass
        
        try:
            cnt0 = contours[two_largest[0]].reshape(-1,2)
            epsilon0 = 0.001*cv2.arcLength(cnt0,True)
            approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
        except:
            pass
        try: 
            cnt1 = contours[two_largest[1]].reshape(-1,2)
            epsilon1 = 0.001*cv2.arcLength(cnt1,True)
            approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
        except:
            pass

        canvas = np.zeros(img.shape, np.uint8)
            
        del img
        gc.collect()
        
        try:
            cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
        except:
            pass
        try:
            cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
        except:
            pass
        canvas = canvas[:,:]
        list_prediction.append(canvas)

    list_prediction = np.expand_dims(list_prediction, axis=-1)
    gc.collect()
    return list_prediction

def make_generator_model():
    
    input_shape=(SIZE,SIZE,CHANNELS)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
       
    return model

generator = make_generator_model()



model1 = generator
model1.load_weights('../input/gan-segmenter-100-bs8-27-bs1/generator_model_segmentation_100_27_epochs.h5')
model2 = load_model('../input/autoencoders-segmentation-300-epochs/xnet_segmenter_300_epochs.h5', compile=False)
model3 = load_model('../input/autoencoders-segmentation-300-epochs/resnet34_segmenter_300_epochs.h5', compile=False)

models = [model1, model2, model3]

X_data1 = [np.asarray(IMAGE_DISEASE_1275),np.asarray(IMAGE_DISEASE_OTSU),np.asarray(IMAGE_DISEASE_CANNY)]

del IMAGE_DISEASE_1275
del IMAGE_DISEASE_OTSU
del IMAGE_DISEASE_CANNY
gc.collect()

X_data2 = np.asarray(IMAGE_DISEASE_2550)
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
X_data3 = preprocess_input1(np.asarray(IMAGE_DISEASE_2550))


pred1 = model1.predict(X_data1)
del X_data1
gc.collect()

pred2 = model2.predict(X_data2)
del X_data2
gc.collect()

pred3 = model3.predict(X_data3)
del X_data3
gc.collect()

prediction1 = list()
prediction2 = list()
prediction3 = list()

predi1 = processing(pred1,prediction1)
del pred1
gc.collect()

predi2 = processing(pred2,prediction2)
del pred2
gc.collect()

predi3 = processing(pred3,prediction3)
del pred3
gc.collect()

preds=np.asarray([predi1, predi2, predi3])
preds.astype(np.float16)


del predi1
del predi2
del predi3
gc.collect()

wted_preds = np.tensordot(preds, [0.2, 0.3, 0.3], axes=((0),(0)))
wted_ensemble_pred = np.squeeze(wted_preds.astype(np.float16))
del wted_preds
del preds
gc.collect()

wted = list()
for l,ima in enumerate(tqdm(wted_ensemble_pred)):

    out = np.zeros_like(ima)
    for i in range(0,SIZE):
        lim = 0.75*ima.max()
        for j in range(0,SIZE):
            if ima[i,j] >= lim:
                out[i,j] = 255
    wted.append(out)
    
wted_images = np.array(wted)
wted_images = np.expand_dims(wted_images, axis=-1)

In [ ]:
os.makedirs('./Diseases/masks')
for i in tqdm(range(0+c,(int(len(NAMES_JPG_1)/16)+c))):
    predicted_rx = np.squeeze(wted_images[i-c].astype(np.float32))
    cv2.imwrite('./Diseases/masks/'+ str(NAMES_JPG_1.iloc[i]),predicted_rx)

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./DISEASES_MASKS_15.zip', 'w')
 
for folder, subfolders, files in os.walk('./Diseases'):
 
    for file in files:
        if file.endswith('.jpg'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
#https://www.kaggle.com/vbookshelf/play-audio-read-the-files-create-a-spectrogram

from pydub import AudioSegment
import IPython

path = '../input/alarm-sound/Alarm_Slow_A1_fesliyanstudios.mp3'

IPython.display.Audio(path, autoplay=True)

# 5.  Defining the metrics for the models

In [ ]:
def iou_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def dice_coef2(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    if union==0: return 1
    intersection = np.sum(y_true_f * y_pred_f)
    return 2. * intersection / union

def mean_dice_coef(y_true, y_pred_bin):
    batch_size = y_true.shape[0]
    channel_num = y_true.shape[-1]
    mean_dice_channel = 0.
    for i in range(batch_size):
        for j in range(channel_num):
            channel_dice = dice_coef2(y_true[i, :, :, j], y_pred_bin[i, :, :, j])
            mean_dice_channel += channel_dice/(channel_num*batch_size)
    return mean_dice_channel

# 6. Bone supression from Chest X-Ray using Autoencoder and TPU

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
 with tpu_strategy.scope():
    input_shape=(SIZE,SIZE,1)
    classes=1
    kernel_size = 3
    filter_depth = (64*2,128*2,256*2,512*2,0*2)

    img_input = Input(shape=input_shape)

    conv1 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(img_input)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation("relu")(batch1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(act1)

    conv2 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(pool1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation("relu")(batch2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(act2)

    conv3 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation("relu")(batch3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(act3)

    conv4 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation("relu")(batch4)

    conv5 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation("relu")(batch5)

    up6 = UpSampling2D(size=(2, 2))(act5)
    conv6 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up6)
    batch6 = BatchNormalization()(conv6)
    act6 = Activation("relu")(batch6)
    concat6 = concatenate([act3,act6])

    up7 = UpSampling2D(size=(2, 2))(concat6)
    conv7 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up7)
    batch7 = BatchNormalization()(conv7)
    act7 = Activation("relu")(batch7)
    concat7 = concatenate([act2,act7])

    conv8 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = Activation("relu")(batch8)
    pool8 = MaxPooling2D(pool_size=(2, 2))(act8)

    conv9 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool8)
    batch9 = BatchNormalization()(conv9)
    act9 = Activation("relu")(batch9)
    pool9 = MaxPooling2D(pool_size=(2, 2))(act9)

    conv10 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool9)
    batch10 = BatchNormalization()(conv10)
    act10 = Activation("relu")(batch10)

    conv11 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = Activation("relu")(batch11)

    up12 = UpSampling2D(size=(2, 2))(act11)
    conv12 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up12)
    batch12 = BatchNormalization()(conv12)
    act12 = Activation("relu")(batch12)
    concat12 = concatenate([act9,act12])

    up13 = UpSampling2D(size=(2, 2))(concat12)
    conv13 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up13)
    batch13 = BatchNormalization()(conv13)
    act13 =  Activation("relu")(batch13)
    concat13 = concatenate([act8,act13])

    up14 = UpSampling2D(size=(2, 2))(concat13)
    conv14 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(up14)
    batch14 = BatchNormalization()(conv14)
    act14 = Activation("relu")(batch14)
    concat14 = concatenate([act1,act14])

    output_xnet = Conv2D(1, (1, 1), activation='sigmoid')(concat14)

    model = Model(img_input, output_xnet)
    model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=[dice_coef_loss], steps_per_execution=32)
    model.summary()

In [ ]:
# https://www.kaggle.com/bochrabenjaballah/petals-to-the-metal-flower-classification-on-tpu
# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.000001, max_lr = 0.0001,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.982):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

In [ ]:
EPOCHS = 300
rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
callbacks = [lr_callback, ModelCheckpoint('Supression_lung_Xnet.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
history = model.fit(X_train, y_train, batch_size = 16 * tpu_strategy.num_replicas_in_sync, epochs = 300, verbose = 1, validation_data = (X_test,y_test), callbacks = callbacks)

In [ ]:
model.save('Autoencoder_300_Xnet_supression_LearningRateScheduler.h5')

In [ ]:
 #with tpu_strategy.scope(): 
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model_autoencoder = load_model('../input/autoencoder-supression-300-epochs/Autoencoder_300_Xnet_supression_LearningRateScheduler (1).h5', custom_objects=dependencies)
loaded_model_autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
    #loaded_model_autoencoder.evaluate(X_test, y_test, batch_size = 16 * tpu_strategy.num_replicas_in_sync)

# 7. Gan for semantic segmentationfrom Chest X-Ray

**Generator architecture** adapted from [link](https://github.com/JosephPB/XNet).

In [ ]:
import tensorflow as tf  # TF 2.0


class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)

In [ ]:
from keras.initializers import RandomNormal
def make_generator_model():
    
    input_shape=(SIZE,SIZE,CHANNELS)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
       
    return model

In [ ]:
generator = make_generator_model()
generator.summary()

In [ ]:
def make_discrimator_model():
    
    init = RandomNormal(mean = 0.0, stddev = 0.02)
    inp = Input(shape=[SIZE, SIZE, 1], name='input_image')
    inp_otsu = Input(shape=[SIZE, SIZE, 1], name='input_image_otsu')
    inp_canny = Input(shape=[SIZE, SIZE, 1], name='input_image_canny')
    tar = Input(shape=[SIZE, SIZE, 1], name='target_image')
    
    merged = tf.keras.layers.concatenate([inp,inp_otsu,inp_canny,tar])
                                    
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    flat = layers.Flatten()(d)
    patch_out = layers.Dense(1, activation='sigmoid')(flat) 
    model = Model(inputs=[inp,inp_otsu,inp_canny, tar], outputs=patch_out)
    
    return model

In [ ]:
discriminator = make_discrimator_model()
discriminator.summary()

In [ ]:
import shutil

shutil.rmtree('./input/training_checkpoints')
shutil.rmtree('./input/images_gan')

In [ ]:
os.makedirs('./input/training_checkpoints')

MAE = tf.keras.losses.MeanAbsoluteError()
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = binary_cross_entropy(tf.multiply(tf.ones_like(disc_real_output),0.9), disc_real_output)
    fake_loss = binary_cross_entropy(tf.multiply(tf.zeros_like(disc_generated_output),0.1), disc_generated_output)
    total_disc_loss = real_loss + fake_loss
    return total_disc_loss

def generator_loss(disc_generated_output,fake_output_,real_output_):
    gan_loss = binary_cross_entropy(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = MAE(real_output_,fake_output_)
    total_gen_loss = gan_loss + (100 * l1_loss) 
    return total_gen_loss, gan_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

checkpoint_dir = './input/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
epochs = 1
num_examples_to_generate = 16
idx = np.random.randint(0, len(SEGMENTATION_IMAGES_GAN_3),num_examples_to_generate)
seed1 = SEGMENTATION_IMAGES_GAN_3[idx]
seed2 = SEGMENTATION_IMAGES_OTSU_GAN_3[idx]
seed3 = SEGMENTATION_IMAGES_CANNY_GAN_3[idx]

In [ ]:
import datetime
log_dir="../logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
def train_steps(input_image,input_image_otsu,input_image_canny,target, epoch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([input_image,input_image_otsu,input_image_canny], training=True)
        real_output = discriminator([input_image,input_image_otsu,input_image_canny, target], training=True)
        fake_output = discriminator([input_image,input_image_otsu,input_image_canny, generated_images], training=True)
        gen_total_loss, gen_gan_loss = generator_loss(fake_output, generated_images, target)
        disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    with summary_writer.as_default():
        tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
        tf.summary.scalar('disc_loss', disc_loss, step=epoch)
    return gen_total_loss,disc_loss

In [ ]:
os.makedirs('./input/images_gan')
def generate_and_save_images(model, epoch, test_input,test_input2,test_input3):
    predictions = model([test_input,test_input2,test_input3],training = False)

    fig = plt.figure(figsize=(8,8))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.savefig('./input/images_gan/image_at_epoch_{:04d}.png'.format(epoch))   
    plt.show()

In [ ]:
def plot_loss(epoch, g_losses, d_losses):
    plt.figure(figsize=(10,5))
    plt.title("Loss, Epochs 0-" + str(epoch))
    plt.plot(g_losses,label="Generator")
    plt.plot(d_losses,label="Discriminator")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
import numpy

def train(X_dataset,X_train_dataset_otsu,X_train_dataset_canny,y_dataset, epochs):
    g_loss = []
    d_loss = []
    for epoch in range(epochs):
        start = time.time()
        
        for image_batch,image_batch_otsu,image_batch_canny,target_batch in zip(X_dataset,X_train_dataset_otsu,X_train_dataset_canny,y_dataset):
            generator_loss, discriminator_loss = train_steps(image_batch,image_batch_otsu,image_batch_canny,target_batch, epochs)

        g_loss.append(generator_loss.numpy())
    
        d_loss.append(discriminator_loss.numpy())
        
        display.clear_output(wait = True)
        print("Generator loss: " + str(g_loss))
        print("Discriminator loss: " + str(d_loss))
        plot_loss(epoch, g_loss, d_loss)
        generate_and_save_images(generator, epoch + 1, seed1,seed2,seed3)

        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))

In [ ]:
def display_image(epoch_no):
    return PIL.Image.open('./input/images_gan/image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
train(train_dataset_X,train_dataset_otsu_X,train_dataset_canny_X,train_dataset_y, epochs)

In [ ]:
dice = list()
eee = list()
for e in range(1,26):
    ee = 100 + e
    eee.append(ee)
    train(train_dataset_X,train_dataset_otsu_X,train_dataset_canny_X,train_dataset_y, epochs)
    predictions = generator.predict([SEGMENTATION_IMAGES_GAN_3,SEGMENTATION_IMAGES_OTSU_GAN_3,SEGMENTATION_IMAGES_CANNY_GAN_3])
    dice.append(mean_dice_coef((SEGMENTATION_TARGETS_GAN_3*127.5 + 127.5)/255, (predictions*127.5 + 127.5)/255))
    print('Mean dice coef epoch ' + str(ee) + ': ' + str(dice))
    generator.save('generator_model_segmentation_'+ str(ee) +'_epochs.h5')
    discriminator.save('discriminartor_segmentation_'+ str(ee) +'_epochs.h5')
    

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
import PIL
display_image(20)

**GIF :)**

In [ ]:
import glob
import imageio

anim_file = 'gan_segmentation.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('./input/images_gan/image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs.vis.embed as embed

embed.embed_file(anim_file)

**Save and load the model**

In [ ]:
generator.save('generator_model_segmentation_100_27_epochs.h5')
discriminator.save('discriminartor_segmentation_100_27_epochs.h5')

In [ ]:
#https://www.kaggle.com/vbookshelf/play-audio-read-the-files-create-a-spectrogram

from pydub import AudioSegment
import IPython

path = '../input/alarm-sound/Alarm_Slow_A1_fesliyanstudios.mp3'

IPython.display.Audio(path, autoplay=True)

In [ ]:
generator.load_weights('../input/gan-segmenter-100-bs8-25-bs1/generator_model_segmentation_100_25_epochs.h5')
discriminator.load_weights('../input/gan-segmenter-100-bs8-25-bs1/discriminartor_segmentation_100_25_epochs.h5')

**Show and save images**

Train

In [ ]:
number = 300
image_rx = np.squeeze(...[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

predicted_rx = generator.predict([...[[number]].astype(np.float64),...[[number]].astype(np.float64),...[[number]].astype(np.float64)],verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)

img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

cnt0 = contours[0].reshape(-1,2)
epsilon0 = 0.000001*cv2.arcLength(cnt0,True)
approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
try: 
    cnt1 = contours[1].reshape(-1,2)
    epsilon1 = 0.0000001*cv2.arcLength(cnt1,True)
    approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
except:
    pass
canvas = np.zeros(thresh.shape, np.uint8)

cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
try:
    cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
except:
    pass
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('predicted_000.png',thresh)
predicted_000 = cv2.imread('./predicted_000.png',1)

out = np.zeros_like(thresh)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if thresh[i,j] == 255:
            out[i,j] = image_rx[i,j]*127.5 + 127.5
        
cv2.imwrite('out_000.png',out)
out_000 = cv2.imread('./out_000.png',1)

stack = np.hstack((image_000, predicted_000,out_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

Test

In [ ]:
import heapq

number = 30
image_rx = np.squeeze(SEGMENTATION_IMAGES_GAN_3[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

predicted_rx = generator.predict([SEGMENTATION_IMAGES_GAN_3[[number]].astype(np.float64),SEGMENTATION_IMAGES_OTSU_GAN_3[[number]].astype(np.float64),SEGMENTATION_IMAGES_CANNY_GAN_3[[number]].astype(np.float64)],verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)

img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

len_contours = list()
for i in range(0,len(contours)):
    len_contours.append(len(contours[i]))

#two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
two_largest, vals = zip(*data)

cnt0 = contours[two_largest[0]].reshape(-1,2)
epsilon0 = 0.001*cv2.arcLength(cnt0,True)
approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
try: 
    cnt1 = contours[two_largest[1]].reshape(-1,2)
    epsilon1 = 0.001*cv2.arcLength(cnt1,True)
    approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
except:
    pass

canvas = np.zeros(thresh.shape, np.uint8)
cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

try:
    cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
except:
    pass
#ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('predicted_000.png',canvas)
predicted_000 = cv2.imread('./predicted_000.png',1)

out = np.zeros_like(thresh)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if canvas[i,j] == 255:
            out[i,j] = image_rx[i,j]*127.5 + 127.5
        
cv2.imwrite('out_000.png',out)
out_000 = cv2.imread('./out_000.png',1)

stack = np.hstack((image_000, predicted_000,out_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

**Analyzing the mean dice coefficient for test dataset**

**Let's transform our supressed pneumothorax dataset into segmented pneumothorax dataset**

# 8. Autoencoders for semantic segmentation from Chest X-Ray

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    input_shape=(SIZE,SIZE,1)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)

    img_input = Input(shape=input_shape)

    conv1 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(img_input)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation("relu")(batch1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(act1)

    conv2 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(pool1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation("relu")(batch2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(act2)

    conv3 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation("relu")(batch3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(act3)

    conv4 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation("relu")(batch4)

    conv5 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation("relu")(batch5)

    up6 = UpSampling2D(size=(2, 2))(act5)
    conv6 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up6)
    batch6 = BatchNormalization()(conv6)
    act6 = Activation("relu")(batch6)
    concat6 = concatenate([act3,act6])

    up7 = UpSampling2D(size=(2, 2))(concat6)
    conv7 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up7)
    batch7 = BatchNormalization()(conv7)
    act7 = Activation("relu")(batch7)
    concat7 = concatenate([act2,act7])

    conv8 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = Activation("relu")(batch8)
    pool8 = MaxPooling2D(pool_size=(2, 2))(act8)

    conv9 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool8)
    batch9 = BatchNormalization()(conv9)
    act9 = Activation("relu")(batch9)
    pool9 = MaxPooling2D(pool_size=(2, 2))(act9)

    conv10 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool9)
    batch10 = BatchNormalization()(conv10)
    act10 = Activation("relu")(batch10)

    conv11 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = Activation("relu")(batch11)

    up12 = UpSampling2D(size=(2, 2))(act11)
    conv12 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up12)
    batch12 = BatchNormalization()(conv12)
    act12 = Activation("relu")(batch12)
    concat12 = concatenate([act9,act12])

    up13 = UpSampling2D(size=(2, 2))(concat12)
    conv13 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up13)
    batch13 = BatchNormalization()(conv13)
    act13 =  Activation("relu")(batch13)
    concat13 = concatenate([act8,act13])

    up14 = UpSampling2D(size=(2, 2))(concat13)
    conv14 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(up14)
    batch14 = BatchNormalization()(conv14)
    act14 = Activation("relu")(batch14)
    concat14 = concatenate([act1,act14])

    output_xnet = Conv2D(1, (1, 1), activation='sigmoid')(concat14)

    model = Model(img_input, output_xnet)
    model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=[dice_coef_loss])
    model.summary()

In [ ]:
number = 3
fig, ax = plt.subplots(1,2, figsize = (16,8))
ax[0].imshow(SEGMENTATION_IMAGES[number], cmap='gray')
ax[1].imshow(SEGMENTATION_TARGETS[number], cmap='gray')
plt.show()

In [ ]:
callbacks = [ModelCheckpoint('Segmenter_lung_mc_epochs.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
results = model.fit(SEGMENTATION_IMAGES, SEGMENTATION_TARGETS, validation_split=0.1,batch_size=16 * tpu_strategy.num_replicas_in_sync, epochs=300, callbacks = callbacks)

In [ ]:
import heapq
number = 3
image_rx = np.squeeze(SEGMENTATION_IMAGES[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

target_rx = np.squeeze(SEGMENTATION_TARGETS[number].astype(np.float64))
plt.imsave('target_000.jpg', target_rx, cmap='gray')
target_000 = cv2.imread('./target_000.jpg',1)

predicted_rx = model.predict([SEGMENTATION_IMAGES[[number]].astype(np.float64)],verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)

img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

len_contours = list()
for i in range(0,len(contours)):
    len_contours.append(len(contours[i]))

#two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
two_largest, vals = zip(*data)

cnt0 = contours[two_largest[0]].reshape(-1,2)
epsilon0 = 0.001*cv2.arcLength(cnt0,True)
approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
try: 
    cnt1 = contours[two_largest[1]].reshape(-1,2)
    epsilon1 = 0.001*cv2.arcLength(cnt1,True)
    approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
except:
    pass

canvas = np.zeros(thresh.shape, np.uint8)
cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

try:
    cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
except:
    pass
#ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('predicted_000.png',canvas)
predicted_000 = cv2.imread('./predicted_000.png',1)

out = np.zeros_like(thresh)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if canvas[i,j] == 255:
            out[i,j] = image_rx[i,j]*127.5 + 127.5
        
cv2.imwrite('out_000.png',out)
out_000 = cv2.imread('./out_000.png',1)

stack = np.hstack((image_000, target_000, predicted_000,out_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
model.save('xnet_segmenter_300_epochs.h5')

In [ ]:
plt.plot(results.history['dice_coef_loss'], label='train')
plt.plot(results.history['val_dice_coef_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/segmenter-mais-eficente-300/Segmenter_lung (2).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
results = loaded_model.evaluate(x_val,SEGMENTATION_TARGETS_3,batch_size=16, verbose=1)
print("Loss and dice_coef_loss:", results)

Second autoencoder

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    n_classes = 1
    activation = 'sigmoid'
    LR = 0.0001
    optim = keras.optimizers.Adam(LR)
    BATCH_SIZE = 16
    BACKBONE1 = 'resnet34'
    preprocess_input1 = sm.get_preprocessing(BACKBONE1)

    x_train = preprocess_input1(SEGMENTATION_IMAGES)

    base_model = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes = n_classes, activation = activation)

    inp = Input(shape=(None, None, 1))
    l1 = Conv2D(3, (1, 1))(inp)
    out = base_model(l1)

    model = Model(inp, out, name=base_model.name)

    model.compile('Adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
    print(model.summary())

In [ ]:
history = model.fit(x_train, SEGMENTATION_TARGETS, batch_size = 16 * tpu_strategy.num_replicas_in_sync, epochs = 300, verbose = 1)

In [ ]:
model.save('resnet34_segmenter_300_epochs.h5')

In [ ]:
import heapq
number = 4
image_rx = np.squeeze(SEGMENTATION_IMAGES_3[number].astype(np.float64))
plt.imsave('image_000.jpg', image_rx, cmap='gray')
image_000 = cv2.imread('./image_000.jpg',1)

target_rx = np.squeeze(SEGMENTATION_TARGETS_3[number].astype(np.float64))
plt.imsave('target_000.jpg', target_rx, cmap='gray')
target_000 = cv2.imread('./target_000.jpg',1)

predicted_rx = model3.predict([SEGMENTATION_IMAGES_3[[number]].astype(np.float64)],verbose = 1)
predicted_rx = np.squeeze(predicted_rx.astype(np.float64))
#predicted_rx = predicted_rx[0,:,:]
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)

img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((5,5),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

len_contours = list()
for i in range(0,len(contours)):
    len_contours.append(len(contours[i]))

#two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
two_largest, vals = zip(*data)

cnt0 = contours[two_largest[0]].reshape(-1,2)
epsilon0 = 0.001*cv2.arcLength(cnt0,True)
approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
try: 
    cnt1 = contours[two_largest[1]].reshape(-1,2)
    epsilon1 = 0.001*cv2.arcLength(cnt1,True)
    approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
except:
    pass

canvas = np.zeros(thresh.shape, np.uint8)
cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

try:
    cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
except:
    pass
#ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('predicted_000.png',canvas)
predicted_000 = cv2.imread('./predicted_000.png',1)

out = np.zeros_like(thresh)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if canvas[i,j] == 255:
            out[i,j] = image_rx[i,j]*127.5 + 127.5
        
cv2.imwrite('out_000.png',out)
out_000 = cv2.imread('./out_000.png',1)

stack = np.hstack((image_000, target_000, predicted_000,out_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('./Segmenter_lung.h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

# 9. Structuring a combinated decision applied to semantic segmentation

In [ ]:
from keras.initializers import RandomNormal
from keras.metrics import MeanIoU

class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)

def make_generator_model():
    
    input_shape=(SIZE,SIZE,CHANNELS)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
       
    return model

generator = make_generator_model()
generator.summary()

In [ ]:
model1 = generator
model1.load_weights('../input/gan-segmenter-100-bs8-27-bs1/generator_model_segmentation_100_27_epochs.h5')
model2 = load_model('../input/autoencoders-segmentation-300-epochs/xnet_segmenter_300_epochs.h5', compile=False)
model3 = load_model('../input/autoencoders-segmentation-300-epochs/resnet34_segmenter_300_epochs.h5', compile=False)

Ensemble - methodology avaliable on [link](https://github.com/bnsreenu/python_for_microscopists/blob/master/214_multiclass_Unet_sandstone_segm_models_ensemble.py).

In [ ]:
models = [model1, model2, model3]

In [ ]:
X_data1 = [SEGMENTATION_IMAGES_GAN_3.astype(np.float32),SEGMENTATION_IMAGES_OTSU_GAN_3.astype(np.float32),SEGMENTATION_IMAGES_CANNY_GAN_3.astype(np.float32)]

X_data2 = SEGMENTATION_IMAGES_3.astype(np.float32)

BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
X_data3 = preprocess_input1(SEGMENTATION_IMAGES_3)

In [ ]:
pred1 = model1.predict(X_data1)
pred2 = model2.predict(X_data2)
pred3 = model3.predict(X_data3)

In [ ]:
prediction1 = list()
prediction2 = list()
prediction3 = list()

In [ ]:
import heapq

In [ ]:
for predicted_rx in tqdm(pred1):
    predicted_rxx = np.squeeze(predicted_rx.astype(np.float32))
    plt.imsave('predicted_000.jpg', predicted_rxx, cmap='gray')
    img = cv2.imread('./predicted_000.jpg',1)

    img_grey = img[:,:,0]
    min_ = img_grey.min()
    max_ = img_grey.max()
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((5,5),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    len_contours = list()
    for i in range(0,len(contours)):
        len_contours.append(len(contours[i]))

    #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

    data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
    two_largest, vals = zip(*data)

    cnt0 = contours[two_largest[0]].reshape(-1,2)
    epsilon0 = 0.001*cv2.arcLength(cnt0,True)
    approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
    try: 
        cnt1 = contours[two_largest[1]].reshape(-1,2)
        epsilon1 = 0.001*cv2.arcLength(cnt1,True)
        approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
    except:
        pass

    canvas = np.zeros(thresh.shape, np.uint8)
    cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

    try:
        cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    except:
        pass
    canvas = canvas[:,:]
    prediction1.append(canvas)

predi1 = np.expand_dims(prediction1, axis=-1)

In [ ]:
for predicted_rx in tqdm(pred2):
    predicted_rxx = np.squeeze(predicted_rx.astype(np.float32))
    plt.imsave('predicted_000.jpg', predicted_rxx, cmap='gray')
    img = cv2.imread('./predicted_000.jpg',1)

    img_grey = img[:,:,0]
    min_ = img_grey.min()
    max_ = img_grey.max()
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((5,5),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    len_contours = list()
    for i in range(0,len(contours)):
        len_contours.append(len(contours[i]))

    #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

    data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
    two_largest, vals = zip(*data)

    cnt0 = contours[two_largest[0]].reshape(-1,2)
    epsilon0 = 0.001*cv2.arcLength(cnt0,True)
    approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
    try: 
        cnt1 = contours[two_largest[1]].reshape(-1,2)
        epsilon1 = 0.001*cv2.arcLength(cnt1,True)
        approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
    except:
        pass

    canvas = np.zeros(thresh.shape, np.uint8)
    cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

    try:
        cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    except:
        pass
    canvas = canvas[:,:]
    prediction2.append(canvas)

predi2 = np.expand_dims(prediction2, axis=-1)

In [ ]:
p = 0
for predicted_rx in tqdm(pred3):
    predicted_rxx = np.squeeze(predicted_rx.astype(np.float32))
    plt.imsave('predicted_000.jpg', predicted_rxx, cmap='gray')
    img = cv2.imread('./predicted_000.jpg',1)

    img_grey = img[:,:,0]
    min_ = img_grey.min()
    max_ = img_grey.max()
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((5,5),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    len_contours = list()
    for i in range(0,len(contours)):
        len_contours.append(len(contours[i]))

    #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

    data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
    two_largest, vals = zip(*data)

    cnt0 = contours[two_largest[0]].reshape(-1,2)
    epsilon0 = 0.001*cv2.arcLength(cnt0,True)
    approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
    try: 
        cnt1 = contours[two_largest[1]].reshape(-1,2)
        epsilon1 = 0.001*cv2.arcLength(cnt1,True)
        approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
    except:
        pass

    canvas = np.zeros(thresh.shape, np.uint8)
    cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

    try:
        cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
    except:
        pass
    canvas = canvas[:,:]
    prediction3.append(canvas)
    
predi3 = np.expand_dims(prediction3, axis=-1)

In [ ]:
preds=np.array([predi1, predi2, predi3])

In [ ]:
preds.astype(np.float32)

In [ ]:
"""
wted_preds = np.tensordot(preds, [0.0, 0.0, 0.1], axes=((0),(0)))
wted_ensemble_pred = np.squeeze(wted_preds.astype(np.float32))
wted = list()
ima = wted_ensemble_pred[2]

out = np.zeros_like(ima)
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if ima[i,j] >= 0.75*ima.max():
            out[i,j] = 255.0
wted.append(out)
"""

number = 0
#wted_ensemble_pred = np.argwhere(wted_preds==wted_preds.max())
#wted_ensemble_pred = np.argmax(wted_preds, axis=3)
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].imshow(np.array(SEGMENTATION_TARGETS_3[:,:,:,0][number]*255).astype(np.float32), cmap='gray')
ax[1].imshow(np.array(wted[number]), cmap='gray')
plt.show()

In [ ]:
def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())

In [ ]:
gc.collect()

In [ ]:
import pandas as pd
df = pd.DataFrame([])
i = 0
for w3 in range(0, 4):
    for w2 in range(0,4):
        for w1 in range(0,4):
            wts = [w1/10.,w2/10.,w3/10.]
            if (w1 != 3) or (w2 != 3) or (w3 != 3):
                pass
            else:
                wted_preds = np.tensordot(preds, wts, axes=((0),(0)))
                wted_ensemble_pred = np.squeeze(wted_preds.astype(np.float32))

                wted = list()
                for ima in tqdm(wted_ensemble_pred):

                    out = np.zeros_like(ima)
                    for i in range(0,SIZE):
                        for j in range(0,SIZE):
                            if ima[i,j] >= 0.75*ima.max():
                                out[i,j] = 255.0
                    wted.append(out)

                Xtrue = np.array(SEGMENTATION_TARGETS_3[:,:,:,0]*255.0)
                Xtrue = np.expand_dims(Xtrue, axis=-1)
                Xpred = np.array(wted)
                Xpred = np.expand_dims(Xpred, axis=-1)
                Xtrue /= 255.
                Xpred /= 255.
                print("Now predciting for weights :", w1/10., w2/10., w3/10., " : DICE = ", mean_dice_coef(Xtrue,Xpred))
                df = df.append(pd.DataFrame({'wt1':wts[0],'wt2':wts[1], 
                                             'wt3':wts[2], 'DICE': mean_dice_coef(Xtrue,Xpred)}, index=[0]), ignore_index=True)
            i+=1
            
max_dice_row = df.iloc[df['DICE'].idxmax()]
print("Max DICE of ", max_dice_row[3], " obained with w1=", max_dice_row[0],
      " w2=", max_dice_row[1], " and w3=", max_dice_row[2]) 

Loading images

In [ ]:
auto_train_dir = os.path.join("../input/p-i-3000-500/images_segment_pneumothorax_train_auto/")
auto_test_dir = os.path.join("../input/p-i-3000-500/images_segment_pneumothorax_test_auto/")
auto_train_list = os.listdir(auto_train_dir)
auto_test_list = os.listdir(auto_test_dir)

GAN_train_dir = os.path.join("../input/p-i-3000-500/images_segment_pneumothorax_train_GAN/")
GAN_train_otsu_dir = os.path.join("../input/p-i-3000-500/images_otsu_segment_pneumothorax_train_GAN/")                             
GAN_train_canny_dir = os.path.join("../input/p-i-3000-500/images_canny_segment_pneumothorax_train_GAN/")
GAN_test_dir = os.path.join("../input/p-i-3000-500/images_segment_pneumothorax_test_GAN/")
GAN_test_otsu_dir = os.path.join("../input/p-i-3000-500/images_segment_otsu_pneumothorax_test_GAN/")                             
GAN_test_canny_dir = os.path.join("../input/p-i-3000-500/images_segment_canny_pneumothorax_test_GAN/")
GAN_train_list = os.listdir(GAN_train_dir)
GAN_train_otsu_list = os.listdir(GAN_train_otsu_dir)                          
GAN_train_canny_list = os.listdir(GAN_train_canny_dir)
GAN_test_list = os.listdir(GAN_test_dir)
GAN_test_otsu_list = os.listdir(GAN_test_otsu_dir)                            
GAN_test_canny_list = os.listdir(GAN_test_canny_dir)

In [ ]:
PNEUMOTHORAX_IMAGES_TRAIN_ = list()
PNEUMOTHORAX_IMAGES_TRAIN_GAN_ = list()
PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN_ = list()
PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN_ = list()

for item in tqdm(auto_train_list):
    dir_auto = auto_train_dir + item
    dir_GAN = GAN_train_dir + item
    dir_GAN_otsu = GAN_train_otsu_dir + item
    dir_GAN_canny = GAN_train_canny_dir + item
    
    PNEUMOTHORAX_IMAGES_TRAIN_.append((cv2.imread(dir_auto,0).reshape((256, 256, 1)).astype('float16')) / 255.)
    
    PNEUMOTHORAX_IMAGES_TRAIN_GAN_.append((cv2.imread(dir_GAN,0).reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5)
    
    PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN_.append((cv2.imread(dir_GAN_otsu,0).reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5)
    
    PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN_.append((cv2.imread(dir_GAN_canny,0).reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5)
    
import gc
gc.collect()

In [ ]:
PNEUMOTHORAX_IMAGES_TRAIN = np.asarray(PNEUMOTHORAX_IMAGES_TRAIN_)
PNEUMOTHORAX_IMAGES_TRAIN_GAN = np.asarray(PNEUMOTHORAX_IMAGES_TRAIN_GAN_)
PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN = np.asarray(PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN_)
PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN = np.asarray(PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN_)

In [ ]:
del PNEUMOTHORAX_IMAGES_TRAIN_
del PNEUMOTHORAX_IMAGES_TRAIN_GAN_
del PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN_
del PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN_
gc.collect()

In [ ]:
PNEUMOTHORAX_IMAGES_TEST = list()
PNEUMOTHORAX_IMAGES_TEST_GAN = list()
PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN = list()
PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN = list()

for item in tqdm(auto_test_list):
    dir_auto = auto_test_dir + item
    dir_GAN = GAN_test_dir + item
    dir_GAN_otsu = GAN_test_otsu_dir + item
    dir_GAN_canny = GAN_test_canny_dir + item
    
    im_auto = cv2.imread(dir_auto,0)
    im_auto = (im_auto.reshape((256, 256, 1)).astype('float16')) / 255.
    
    im_GAN = cv2.imread(dir_GAN,0)
    im_GAN = (im_GAN.reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5
    
    im_GAN_otsu = cv2.imread(dir_GAN_otsu,0)
    im_GAN_otsu = (im_GAN_otsu.reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5
    
    im_GAN_canny = cv2.imread(dir_GAN_canny,0)
    im_GAN_canny = (im_GAN_canny.reshape((256, 256, 1)).astype('float16') - 127.5) / 127.5
    
    PNEUMOTHORAX_IMAGES_TEST.append(im_auto)
    PNEUMOTHORAX_IMAGES_TEST_GAN.append(im_GAN)
    PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN.append(im_GAN_otsu)
    PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN.append(im_GAN_canny)
    
import gc
del im_auto
del im_GAN
del im_GAN_otsu
del im_GAN_canny
gc.collect()

In [ ]:
PNEUMOTHORAX_IMAGES_TEST = np.asarray(PNEUMOTHORAX_IMAGES_TEST)
PNEUMOTHORAX_IMAGES_TEST_GAN = np.asarray(PNEUMOTHORAX_IMAGES_TEST_GAN)
PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN = np.asarray(PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN)
PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN = np.asarray(PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN)

In [ ]:
from keras.initializers import RandomNormal
from keras.metrics import MeanIoU
import heapq

class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)

def processing(prediction_,list_prediction):   
    for predicted_rx in tqdm(prediction_):
        predicted_rxx = np.squeeze(predicted_rx.astype(np.float16))
        plt.imsave('predicted_000.jpg', predicted_rxx, cmap='gray')
        img = cv2.imread('./predicted_000.jpg',1)

        img_grey = img[:,:,0]
        min_ = img_grey.min()
        max_ = img_grey.max()
        ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

        len_contours = list()
        for i in range(0,len(contours)):
            len_contours.append(len(contours[i]))

        #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  
        try:
            data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
            two_largest, vals = zip(*data)
        except:
            pass
        
        try:
            cnt0 = contours[two_largest[0]].reshape(-1,2)
            epsilon0 = 0.001*cv2.arcLength(cnt0,True)
            approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
        except:
            pass
        try: 
            cnt1 = contours[two_largest[1]].reshape(-1,2)
            epsilon1 = 0.001*cv2.arcLength(cnt1,True)
            approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
        except:
            pass

        canvas = np.zeros(thresh.shape, np.uint8)
        try:
            cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
        except:
            pass
        try:
            cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
        except:
            pass
        canvas = canvas[:,:]
        list_prediction.append(canvas)

    list_prediction = np.expand_dims(list_prediction, axis=-1)
    gc.collect()
    return list_prediction

def make_generator_model():
    
    input_shape=(SIZE,SIZE,CHANNELS)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
       
    return model

generator = make_generator_model()



model1 = generator
model1.load_weights('../input/gan-segmenter-100-bs8-27-bs1/generator_model_segmentation_100_27_epochs.h5')
model2 = load_model('../input/autoencoders-segmentation-300-epochs/xnet_segmenter_300_epochs.h5', compile=False)
model3 = load_model('../input/autoencoders-segmentation-300-epochs/resnet34_segmenter_300_epochs.h5', compile=False)

models = [model1, model2, model3]

X_data1 = [PNEUMOTHORAX_IMAGES_TEST_GAN.astype(np.float16),PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN.astype(np.float16),PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN.astype(np.float16)]
X_data2 = PNEUMOTHORAX_IMAGES_TEST.astype(np.float16)
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
X_data3 = preprocess_input1(PNEUMOTHORAX_IMAGES_TEST)

pred1 = model1.predict(X_data1)
pred2 = model2.predict(X_data2)
pred3 = model3.predict(X_data3)

prediction1 = list()
prediction2 = list()
prediction3 = list()

predi1 = processing(pred1,prediction1)
predi2 = processing(pred2,prediction2)
predi3 = processing(pred3,prediction3)

preds=np.asarray([predi1, predi2, predi3])
preds.astype(np.float16)

gc.collect()

wted_preds = np.tensordot(preds, [0.2, 0.3, 0.3], axes=((0),(0)))
wted_ensemble_pred = np.squeeze(wted_preds.astype(np.float16))

wted = list()
for ima in tqdm(wted_ensemble_pred):

    out = np.zeros_like(ima)
    for i in range(0,SIZE):
        lim = 0.75*ima.max()
        for j in range(0,SIZE):
            if ima[i,j] >= lim:
                out[i,j] = 255.0
    wted.append(out)
    
wted_images = np.array(wted)
wted_images = np.expand_dims(wted_images, axis=-1)

In [ ]:
number = 30
fig, ax = plt.subplots(1,3, figsize = (8,4))
ax[0].imshow(np.array(PNEUMOTHORAX_IMAGES_TEST[number]).astype(np.float32), cmap='gray')
ax[1].imshow(np.array(wted_images[number]), cmap='gray')
out = np.zeros_like(wted_images[number])
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if wted_images[number][i,j] == 255:
            out[i,j] = PNEUMOTHORAX_IMAGES_TEST[number][i,j]
ax[2].imshow(np.array(out), cmap='gray')
plt.show()

In [ ]:
os.makedirs('./PNEUMOTHORAX/masks_pneumothorax_test')
for i in tqdm(range(0,len(wted_images))):
    predicted_rx = np.squeeze(wted_images[i].astype(np.float32))
    plt.imsave('./PNEUMOTHORAX/masks_pneumothorax_test/'+ auto_test_list[i] + '_mask.png',predicted_rx, cmap = 'gray')

In [ ]:
del out
del ima
del wted_preds
del wted_ensemble_pred
del preds
del pred1
del pred2
del pred3
del predi1
del predi2
del predi3
del wted_images
del X_data1
del X_data2
del X_data3
del PNEUMOTHORAX_IMAGES_TEST_GAN
del PNEUMOTHORAX_IMAGES_TEST
del PNEUMOTHORAX_IMAGES_OTSU_TEST_GAN
del PNEUMOTHORAX_IMAGES_CANNY_TEST_GAN
gc.collect()

In [ ]:
from keras.initializers import RandomNormal
from keras.metrics import MeanIoU
import heapq

class SpectralNormalization(tf.keras.layers.Wrapper):
    def __init__(self, layer, iteration=1, eps=1e-12, training=True, **kwargs):
        self.iteration = iteration
        self.eps = eps
        self.do_power_iteration = training
        if not isinstance(layer, tf.keras.layers.Layer):
            raise ValueError(
                'Please initialize `TimeDistributed` layer with a '
                '`Layer` instance. You passed: {input}'.format(input=layer))
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        self.layer.build(input_shape)

        self.w = self.layer.kernel
        self.w_shape = self.w.shape.as_list()

        self.v = self.add_weight(shape=(1, self.w_shape[0] * self.w_shape[1] * self.w_shape[2]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_v',
                                 dtype=tf.float32)

        self.u = self.add_weight(shape=(1, self.w_shape[-1]),
                                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
                                 trainable=False,
                                 name='sn_u',
                                 dtype=tf.float32)

        super(SpectralNormalization, self).build()

    def call(self, inputs):
        self.update_weights()
        output = self.layer(inputs)
        self.restore_weights()  # Restore weights because of this formula "W = W - alpha * W_SN`"
        return output
    
    def update_weights(self):
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        
        u_hat = self.u
        v_hat = self.v  # init v vector

        if self.do_power_iteration:
            for _ in range(self.iteration):
                v_ = tf.matmul(u_hat, tf.transpose(w_reshaped))
                v_hat = v_ / (tf.reduce_sum(v_**2)**0.5 + self.eps)

                u_ = tf.matmul(v_hat, w_reshaped)
                u_hat = u_ / (tf.reduce_sum(u_**2)**0.5 + self.eps)

        sigma = tf.matmul(tf.matmul(v_hat, w_reshaped), tf.transpose(u_hat))
        self.u.assign(u_hat)
        self.v.assign(v_hat)

        self.layer.kernel.assign(self.w / sigma)

    def restore_weights(self):
        self.layer.kernel.assign(self.w)

def processing(prediction_,list_prediction):   
    for predicted_rx in tqdm(prediction_):
        predicted_rxx = np.squeeze(predicted_rx.astype(np.float16))
        plt.imsave('predicted_000.jpg', predicted_rxx, cmap='gray')
        img = cv2.imread('./predicted_000.jpg',1)

        img_grey = img[:,:,0]
        min_ = img_grey.min()
        max_ = img_grey.max()
        ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

        len_contours = list()
        for i in range(0,len(contours)):
            len_contours.append(len(contours[i]))

        #two_largest = list(map(len_contours.index, heapq.nlargest(2, len_contours)))  

        data = heapq.nlargest(2, enumerate(len_contours), key=lambda x:x[1])
        two_largest, vals = zip(*data)

        cnt0 = contours[two_largest[0]].reshape(-1,2)
        epsilon0 = 0.001*cv2.arcLength(cnt0,True)
        approx0 = cv2.approxPolyDP(cnt0,epsilon0,True)
        try: 
            cnt1 = contours[two_largest[1]].reshape(-1,2)
            epsilon1 = 0.001*cv2.arcLength(cnt1,True)
            approx1 = cv2.approxPolyDP(cnt1,epsilon1,True)
        except:
            pass

        canvas = np.zeros(thresh.shape, np.uint8)
        cv2.drawContours(canvas, [approx0],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)

        try:
            cv2.drawContours(canvas, [approx1],-1, (255, 255), thickness = -1, lineType = cv2.LINE_AA)
        except:
            pass
        canvas = canvas[:,:]
        list_prediction.append(canvas)

    list_prediction = np.expand_dims(list_prediction, axis=-1)
    gc.collect()
    return list_prediction

def make_generator_model():
    
    input_shape=(SIZE,SIZE,CHANNELS)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)
    init = RandomNormal(mean = 0.0, stddev = 0.02)

    img_input1 = Input(shape=input_shape)
    img_input2 = Input(shape=input_shape)
    img_input3 = Input(shape=input_shape)

    concat_input = tf.keras.layers.concatenate([img_input1,img_input2, img_input3])
    
    conv1 = SpectralNormalization(Conv2D(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat_input)
    batch1 = BatchNormalization()(conv1)
    act1 = layers.LeakyReLU(0.2)(batch1)

    conv2 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act1)
    batch2 = BatchNormalization()(conv2)
    act2 = layers.LeakyReLU(0.2)(batch2)

    conv3 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act2)
    batch3 = BatchNormalization()(conv3)
    act3 = layers.LeakyReLU(0.2)(batch3)

    conv4 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act3)
    batch4 = BatchNormalization()(conv4)
    act4 = layers.LeakyReLU(0.2)(batch4)

    conv5 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = layers.LeakyReLU(0.2)(batch5)

    conv6 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act5)
    batch6 = BatchNormalization()(conv6)
    act6 = layers.LeakyReLU(0.2)(batch6)
    concat6 = concatenate([act2,act6])

    conv7 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat6)
    batch7 = BatchNormalization()(conv7)
    act7 = layers.LeakyReLU(0.2)(batch7)
    concat7 = concatenate([act1,act7])

    conv8 = SpectralNormalization(Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = layers.LeakyReLU(0.2)(batch8)

    conv9 = SpectralNormalization(Conv2D(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act8)
    batch9 = BatchNormalization()(conv9)
    act9 = layers.LeakyReLU(0.2)(batch9)

    conv10 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act9)
    batch10 = BatchNormalization()(conv10)
    act10 = layers.LeakyReLU(0.2)(batch10)

    conv11 = SpectralNormalization(Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same", kernel_initializer = init))(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = layers.LeakyReLU(0.2)(batch11)

    conv12 = SpectralNormalization(Conv2DTranspose(filter_depth[2], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(act11)
    batch12 = BatchNormalization()(conv12)
    act12 = layers.LeakyReLU(0.2)(batch12)
    concat12 = concatenate([act9,act12])

    conv13 = SpectralNormalization(Conv2DTranspose(filter_depth[1], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat12)
    batch13 = BatchNormalization()(conv13)
    act13 =  layers.LeakyReLU(0.2)(batch13)
    concat13 = concatenate([act8,act13])

    conv14 = SpectralNormalization(Conv2DTranspose(filter_depth[0], (kernel_size, kernel_size),strides = (2,2), padding="same", kernel_initializer = init))(concat13)
    batch14 = BatchNormalization()(conv14)
    act14 = layers.LeakyReLU(0.2)(batch14)
    concat14 = concatenate([concat_input,act14])

    output_xnet = SpectralNormalization(Conv2D(1, (1, 1), activation='tanh'))(concat14)

    model = Model(inputs = [img_input1,img_input2,img_input3], outputs = output_xnet)
       
    return model

generator = make_generator_model()


model1 = generator
model1.load_weights('../input/gan-segmenter-100-bs8-27-bs1/generator_model_segmentation_100_27_epochs.h5')
model2 = load_model('../input/autoencoders-segmentation-300-epochs/xnet_segmenter_300_epochs.h5', compile=False)
model3 = load_model('../input/autoencoders-segmentation-300-epochs/resnet34_segmenter_300_epochs.h5', compile=False)

models = [model1, model2, model3]

pred1 = model1.predict([PNEUMOTHORAX_IMAGES_TRAIN_GAN.astype(np.float16),PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN.astype(np.float16),PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN.astype(np.float16)])
pred2 = model2.predict(PNEUMOTHORAX_IMAGES_TRAIN.astype(np.float16))
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
pred3 = model3.predict(preprocess_input1(PNEUMOTHORAX_IMAGES_TRAIN))

del PNEUMOTHORAX_IMAGES_TRAIN_GAN
del PNEUMOTHORAX_IMAGES_TRAIN
del PNEUMOTHORAX_IMAGES_OTSU_TRAIN_GAN
del PNEUMOTHORAX_IMAGES_CANNY_TRAIN_GAN
gc.collect()

prediction1 = list()
prediction2 = list()
prediction3 = list()

predi1 = processing(pred1,prediction1)
predi2 = processing(pred2,prediction2)
predi3 = processing(pred3,prediction3)

del pred1
del pred2
del pred3
gc.collect()

preds=np.asarray([predi1, predi2, predi3])
preds.astype(np.float16)


del predi1
del predi2
del predi3
gc.collect()

wted_preds = np.tensordot(preds, [0.2, 0.3, 0.3], axes=((0),(0)))
wted_ensemble_pred = np.squeeze(wted_preds.astype(np.float16))

del wted_preds
del preds
gc.collect()

In [ ]:
wted = list()
for ima in tqdm(wted_ensemble_pred):
    lim = 0.75*ima.max()
    out = np.zeros_like(ima)
    for i in range(0,SIZE):
        for j in range(0,SIZE):
            if ima[i,j] >= lim:
                out[i,j] = 255.0
    wted.append(out)
    
wted_images = np.array(wted)
wted_images = np.expand_dims(wted_images, axis=-1)

In [ ]:
number = 30
fig, ax = plt.subplots(1,3, figsize = (8,4))
ax[0].imshow(np.array(PNEUMOTHORAX_IMAGES_TRAIN[number]).astype(np.float32), cmap='gray')
ax[1].imshow(np.array(wted_images[number]), cmap='gray')
out = np.zeros_like(wted_images[number])
for i in range(0,SIZE):
    for j in range(0,SIZE):
        if wted_images[number][i,j] == 255:
            out[i,j] = PNEUMOTHORAX_IMAGES_TRAIN[number][i,j]
ax[2].imshow(np.array(out), cmap='gray')
plt.show()

In [ ]:
os.makedirs('./PNEUMOTHORAX/masks_pneumothorax_train')
for i in tqdm(range(0,len(wted_images))):
    predicted_rx = np.squeeze(wted_images[i].astype(np.float16))
    plt.imsave('./PNEUMOTHORAX/masks_pneumothorax_train/'+ auto_train_list[i] + '_mask.png',predicted_rx, cmap = 'gray')

In [ ]:
import zipfile
       
jungle_zip = zipfile.ZipFile('./images_and_masks.zip', 'w')
jungle_zip.write('./PNEUMOTHORAX', compress_type=zipfile.ZIP_DEFLATED)
 
jungle_zip.close()

# 10. Structuring the radiomics analysis

In [ ]:
auto_train_dir = os.path.join("../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_train/")
auto_test_dir = os.path.join("../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_test/")
auto_train_list = os.listdir(auto_train_dir)
auto_test_list = os.listdir(auto_test_dir)
DATADIR_IMAGES = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_images/'
DF_TEST = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv')
DF_TRAIN = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv')

In [ ]:
test_dir = os.path.join("../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_test/")
test_list = os.listdir(test_dir)
os.makedirs('./PNEUMOTHORAX/masks_pneumothorax_test')
for i in tqdm(range(0,len(test_list))):
    im = cv2.imread('../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_test/'+ test_list[i],cv2.IMREAD_GRAYSCALE)
    im = np.squeeze(im.astype(np.float16))
    plt.imsave('./PNEUMOTHORAX/masks_pneumothorax_test/'+ str(i) + '.jpg',im, cmap = 'gray')

In [ ]:
train_dir = os.path.join("../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_train/")
train_list = os.listdir(train_dir)
os.makedirs('./PNEUMOTHORAX/masks_pneumothorax_train')
for i in tqdm(range(0,len(train_list))):
    im = cv2.imread('../input/pneumothorax-xray-images-and-masks/masks_pneumothorax_train/'+ train_list[i],cv2.IMREAD_GRAYSCALE)
    im = np.squeeze(im.astype(np.float16))
    plt.imsave('./PNEUMOTHORAX/masks_pneumothorax_train/'+ str(i) + '.jpg',im, cmap = 'gray')

In [ ]:
LEN_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0].shape[0]
LEN_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1].shape[0]
LEN_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0].shape[0]
LEN_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1].shape[0]

IMAGE_TRAIN_PNEUMO = list()
IMAGE_TRAIN_NO_PNEUMO = list()
IMAGE_TEST_PNEUMO = list()
IMAGE_TEST_NO_PNEUMO = list()

CLASS_IMAGE_TRAIN_PNEUMO = list()
CLASS_IMAGE_TRAIN_NO_PNEUMO = list()
CLASS_IMAGE_TEST_PNEUMO = list()
CLASS_IMAGE_TEST_NO_PNEUMO = list()

NAME_TRAIN_PNEUMO = list()
NAME_TRAIN_NO_PNEUMO = list()
NAME_TEST_PNEUMO = list()
NAME_TEST_NO_PNEUMO = list()

for i in tqdm(range(0,int(1500))):
    DF_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1]
    IMAGE_NAME = DF_TRAIN_PNEUMO.iloc[i,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_PNEUMO.append(image)
    CLASS_IMAGE_TRAIN_PNEUMO.append(1)
    NAME_TRAIN_PNEUMO.append(IMAGE_NAME)

for j in tqdm(range(0,int(1500))):
    DF_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0]
    IMAGE_NAME = DF_TRAIN_NO_PNEUMO.iloc[j,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TRAIN_NO_PNEUMO.append(image)
    CLASS_IMAGE_TRAIN_NO_PNEUMO.append(0)
    NAME_TRAIN_NO_PNEUMO.append(IMAGE_NAME)
    
for k in tqdm(range(0,int(250))):
    DF_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1]
    IMAGE_NAME = DF_TEST_PNEUMO.iloc[k,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TEST_PNEUMO.append(image)
    CLASS_IMAGE_TEST_PNEUMO.append(1)
    NAME_TEST_PNEUMO.append(IMAGE_NAME)
    
for l in tqdm(range(0,int(250))):
    DF_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0]
    IMAGE_NAME = DF_TEST_NO_PNEUMO.iloc[l,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    IMAGE_TEST_NO_PNEUMO.append(image)
    CLASS_IMAGE_TEST_NO_PNEUMO.append(0)
    NAME_TEST_NO_PNEUMO.append(IMAGE_NAME)

CLASS_IMAGE_TRAIN_PNEUMO = np.array(CLASS_IMAGE_TRAIN_PNEUMO)
CLASS_IMAGE_TRAIN_NO_PNEUMO = np.array(CLASS_IMAGE_TRAIN_NO_PNEUMO)
CLASS_IMAGE_TEST_PNEUMO = np.array(CLASS_IMAGE_TEST_PNEUMO)
CLASS_IMAGE_TEST_NO_PNEUMO = np.array(CLASS_IMAGE_TEST_NO_PNEUMO)

CLASS_TRAIN = np.concatenate((CLASS_IMAGE_TRAIN_PNEUMO,CLASS_IMAGE_TRAIN_NO_PNEUMO))
CLASS_TEST = np.concatenate((CLASS_IMAGE_TEST_PNEUMO,CLASS_IMAGE_TEST_NO_PNEUMO))

IMAGE_TRAIN_PNEUMO = np.array(IMAGE_TRAIN_PNEUMO).reshape((len(IMAGE_TRAIN_PNEUMO), SIZE, SIZE, CHANNELS))
IMAGE_TRAIN_NO_PNEUMO = np.array(IMAGE_TRAIN_NO_PNEUMO).reshape((len(IMAGE_TRAIN_NO_PNEUMO), SIZE, SIZE, CHANNELS))
IMAGE_TEST_PNEUMO = np.array(IMAGE_TEST_PNEUMO).reshape((len(IMAGE_TEST_PNEUMO), SIZE, SIZE, CHANNELS))
IMAGE_TEST_NO_PNEUMO = np.array(IMAGE_TEST_NO_PNEUMO).reshape((len(IMAGE_TEST_NO_PNEUMO), SIZE, SIZE, CHANNELS))

PNEUMOTHORAX_IMAGES_TRAIN = np.concatenate((IMAGE_TRAIN_PNEUMO,IMAGE_TRAIN_NO_PNEUMO))
PNEUMOTHORAX_IMAGES_TEST = np.concatenate((IMAGE_TEST_PNEUMO,IMAGE_TEST_NO_PNEUMO))

In [ ]:
#os.makedirs('./PNEUMOTHORAX/images_pneumothorax_train')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TRAIN))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TRAIN[i].astype(np.float16))
    plt.imsave('./PNEUMOTHORAX/images_pneumothorax_train/'+ str(i) + '.jpg',predicted_rx, cmap = 'gray')
#os.makedirs('./PNEUMOTHORAX/images_pneumothorax_test')
for i in tqdm(range(0,len(PNEUMOTHORAX_IMAGES_TEST))):
    predicted_rx = np.squeeze(PNEUMOTHORAX_IMAGES_TEST[i].astype(np.float16))
    plt.imsave('./PNEUMOTHORAX/images_pneumothorax_test/'+ str(i) + '.jpg',predicted_rx, cmap = 'gray')    

In [ ]:
applyLog = False
applyWavelet = False

settings = {'binWidth':128,
            'label': 1, 
            'interpolator': sitk.sitkBSpline, 
            'resampledPixelSpacing':None, 
            'force2D': True,
            'force2Ddimension': 0}

extractor = featureextractor.RadiomicsFeatureExtractor(additionalInfo=True, **settings)
extractor.enableAllImageTypes()

featureVector_test = []

In [ ]:
for i in range(0,len(PNEUMOTHORAX_IMAGES_TEST)): 
    image_test = sitk.ReadImage('./PNEUMOTHORAX/images_pneumothorax_test/'+ str(i) + '.jpg', sitk.sitkInt8)
    mask_test = sitk.ReadImage('./PNEUMOTHORAX/masks_pneumothorax_test/'+ str(i) + '.jpg', sitk.sitkInt8)
    result_test = pd.Series(extractor.execute(image_test, mask_test))
    featureVector_test.append(result_test)
    clear_output(wait=True)

df_test = pd.DataFrame(featureVector_test)
df_test.to_csv('Features_test', sep=',', encoding='utf-8')
df_test = pd.read_csv('./Features_test')
df_test

In [ ]:
applyLog = False
applyWavelet = False

settings = {'binWidth':128,
            'label': 1, 
            'interpolator': sitk.sitkBSpline, 
            'resampledPixelSpacing':None, 
            'force2D': True,
            'force2Ddimension': 0}

extractor = featureextractor.RadiomicsFeatureExtractor(additionalInfo=True, **settings)
extractor.enableAllImageTypes()

featureVector_train = []

In [ ]:
for i in range(0,len(PNEUMOTHORAX_IMAGES_TRAIN)): 
    image_train = sitk.ReadImage('./PNEUMOTHORAX/images_pneumothorax_train/'+ str(i) + '.jpg', sitk.sitkInt8)
    mask_train = sitk.ReadImage('./PNEUMOTHORAX/masks_pneumothorax_train/'+ str(i) + '.jpg', sitk.sitkInt8)
    result_train = pd.Series(extractor.execute(image_train, mask_train))
    featureVector_train.append(result_train)
    clear_output(wait=True)

df_train = pd.DataFrame(featureVector_train)
df_train.to_csv('Features_train', sep=',', encoding='utf-8')
df_train = pd.read_csv('./Features_train')
df_train

In [ ]:
df_train = pd.read_csv('./Features_train')
df_test = pd.read_csv('./Features_train')

In [ ]:
pd.set_option('display.max_columns', df_train.shape[0]+1)
df_train

In [ ]:
print(df_train.columns.tolist())

In [ ]:
print(df_test.columns.tolist())

In [ ]:
df_train = df_train.drop(columns = ['Unnamed: 0', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'])
df_test = df_test.drop(columns = ['Unnamed: 0', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'])

In [ ]:
pd.set_option('display.max_columns', df_test.shape[0]+1)
df_test

In [ ]:
df_train['class'] = CLASS_TRAIN
df_test['class'] = CLASS_TEST

In [ ]:
count_nan_in_df = df_train.isnull().sum()
df_nan = pd.DataFrame(count_nan_in_df)
pd.set_option('display.max_rows', df_nan.shape[0]+1)
df_nan

In [ ]:
df_train_nonan = df_train.apply (pd.to_numeric, errors='coerce')
df_train_nonan = df_train_nonan.dropna()
df_train_nonan = df_train_nonan.reset_index(drop=True)

In [ ]:
df_test_nonan = df_test.apply (pd.to_numeric, errors='coerce')
df_test_nonan = df_test_nonan.dropna()
df_test_nonan = df_test_nonan.reset_index(drop=True)

In [ ]:
frames = [df_train_nonan, df_test_nonan]
df = pd.concat(frames)

In [ ]:
list_columns = df.columns.tolist()
list_columns = list_columns[0:837]
new = pd.DataFrame(columns=list_columns)
for col in list_columns:
    new = df[df[str(col)] <= np.percentile(df[str(col)],95)]

In [ ]:
y = new['class']
X = new.drop(columns = ['class'])

In [ ]:
X.describe()

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_F_train, y_C_train)

In [ ]:
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
confusion_matrix = confusion_matrix(y_C_test, prediction.round())
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(X_F_train, y_C_train)
model = SelectFromModel(clf, prefit=True)
new = model.transform(X)

In [ ]:
reduced = pd.DataFrame(new)
reduced.to_csv('Reduced.csv', sep=',', encoding='utf-8')
reduced

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(reduced, y, test_size=0.30, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
corr_features = list()
corr_matrix = reduced.corr()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.4:
            colname = corr_matrix.columns[i]
            corr_features.append(str(colname))

In [ ]:
list_corr = list()
for item in corr_features:
    list_corr.append(int(item))

In [ ]:
df_reduced_corr = reduced.drop(columns = list_corr, axis=0)

In [ ]:
df_reduced_corr

In [ ]:
df_reduced_corr.to_csv('Features_reduced_corr', sep=',', encoding='utf-8')

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_reduced_corr, y, test_size=0.30, random_state=42)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(y_C_test, rf.predict(X_F_test)))

# 11. Structuring a deseases detection

In [ ]:
label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES_INTE.head(40198)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')
NAMES_JPG_1

In [ ]:
os.makedirs('./Diseases/attention')
DIR_1 = './images/DISEASES/images/'
DIR_2 = '../input/diseases-masks/Diseases/masks/'
for i in tqdm(range(0,len(NAMES_JPG_1))):
    PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    PATH2 = DIR_2 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),cv2.IMREAD_GRAYSCALE)
    _,IMAGE2 = cv2.threshold(IMAGE2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    for j in range(0,SIZE):
        for k in range(0,SIZE):
            if IMAGE2[j,k] == 255:
                IMAGE2[j,k] = IMAGE1[j,k]
    cv2.imwrite('./Diseases/attention/'+ str(NAMES_JPG_1.iloc[i]),(np.array(IMAGE2).reshape(SIZE, SIZE, 1)).astype('float32'))

In [ ]:
number = 0
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].imshow((np.array(IMAGE1[number]).reshape(SIZE, SIZE, 1)).astype('float32'), cmap='gray')
ax[1].imshow((np.array(IMAGE2[number]).reshape(SIZE, SIZE, 1)).astype('float32'), cmap='gray')
plt.show()

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./DISEASES_ATTENTION.zip', 'w')
 
for folder, subfolders, files in os.walk('./Diseases'):
 
    for file in files:
        if file.endswith('.jpg'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

By here

In [ ]:
DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'
IMAGES_ORIGINAL = list()
IMAGES_COLLIMATED = list()
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    #IMAGES_ORIGINAL.append(IMAGE1)
    PATH2 = DIR_2 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),cv2.IMREAD_GRAYSCALE)
    IMAGES_COLLIMATED.append(IMAGE2)

In [ ]:
#3C
DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'
IMAGES_ORIGINAL = list()
IMAGES_COLLIMATED = list()
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1))
    #IMAGES_ORIGINAL.append(IMAGE1)
    PATH2 = DIR_2 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),cv2.IMREAD_GRAYSCALE)
    IMAGES_COLLIMATED.append(IMAGE2)

In [ ]:
import gc
#del IMAGE1
del IMAGE2
gc.collect()

In [ ]:
number = 0
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].imshow((np.array(IMAGES_ORIGINAL[number]).reshape(SIZE, SIZE, 1)).astype('float32'), cmap='gray')
ax[1].imshow((np.array(IMAGES_COLLIMATED[number]).reshape(SIZE, SIZE, 1)).astype('float32'), cmap='gray')
plt.show()

In [ ]:
DF_LUNG_DISEASES.head(20)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:14]

In [ ]:
#X_train1, X_test1, y_train1, y_test1 = train_test_split(np.array(IMAGES_ORIGINAL), classes, test_size=0.25,random_state=0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(np.array(IMAGES_COLLIMATED), classes, test_size=0.25,random_state=0)

In [ ]:
import gc
#del IMAGES_ORIGINAL
#del IMAGES_COLLIMATED
gc.collect()

In [ ]:
from tensorflow.keras.applications import VGG16
from keras.layers import *

img_size_target = 256
img_input = Input(shape=(img_size_target, img_size_target, 1))
img_conc = Concatenate()([img_input, img_input, img_input])
base_model = VGG16(input_tensor=img_conc,weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

for layer in base_model.layers[:19]:
    layer.trainable = False
model = models.Sequential()
model.add(base_model)
model.add(Flatten())
base_model.summary()
model.summary()

In [ ]:
for i, layer in enumerate(base_model.layers):
    print(i,layer)

In [ ]:
(pd.DataFrame(np.squeeze(np.array(X_feature)))).to_csv('./features_test', sep=',', encoding='utf-8')

In [ ]:
dfj = pd.DataFrame(np.squeeze(np.array(X_feature)))
dfj

In [ ]:
number_ = 4
for num in range(number_,number_+1):
    c = int((len(IMAGES_COLLIMATED)/5)*num)
    X_feature = list()
    for i in tqdm(range(0+c,int((len(IMAGES_COLLIMATED)/5)+c))):
        images_col = (IMAGES_COLLIMATED[i].reshape((1,256, 256,1)).astype('float16')) / 255.
        X_feature.append(model.predict(images_col))
    (pd.DataFrame(np.squeeze(np.array(X_feature)))).to_csv('./features_'+str(num+1)+'_5.csv', sep=',', encoding='utf-8')
    del images_col
    del X_feature
    gc.collect()

In [ ]:
df_total = pd.read_csv('./features_0_5.csv',dtype = np.float16)

In [ ]:
df_total.to_csv('./features_total.csv', sep=',', encoding='utf-8')

In [ ]:
df_total = pd.concat([df_total,pd.read_csv('./features_2_5.csv',dtype = np.float16)], ignore_index=True)
import gc
gc.collect()

In [ ]:
df_total = pd.concat([df_total,pd.read_csv('./features_3_5.csv',dtype = np.float16)], ignore_index=True)
import gc
gc.collect()

In [ ]:
df_total = pd.concat([df_total,pd.read_csv('./features_4_5.csv',dtype = np.float16)], ignore_index=True)
import gc
gc.collect()

In [ ]:
df_total = pd.concat([df_total,pd.read_csv('./features_5_5.csv',dtype = np.float16)], ignore_index=True)
import gc
gc.collect()

In [ ]:
df_total

In [ ]:
df_total.iloc[:,1:-1]

In [ ]:
del images_col
del X_feature
gc.collect()

*New stage - Let's try somethings*

In [ ]:
DATA1 = (pd.read_csv('../input/test-features1/features_1_5.csv',dtype = np.float16,nrows=2)).iloc[:,1:-1]

In [ ]:
DATA1

In [ ]:
target = DF_LUNG_DISEASES['Finding Labels'].head(10000)

In [ ]:
target.value_counts()

In [ ]:
target = pd.DataFrame(classes[0:10000])
target

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(DATA, target)
model = SelectFromModel(clf, prefit=True)
new = model.transform(DATA)

reduced = pd.DataFrame(new)
reduced.to_csv('Reduced.csv', sep=',', encoding='utf-8')
reduced

In [ ]:
feature_idx = model.get_support()
feature_name = DATA.columns[feature_idx]
feature_name

In [ ]:
for i in tqdm(range(0,13284)):
    reduced.rename(columns={i: feature_name[i]}, inplace = True)
reduced

In [ ]:
reduced

In [ ]:
corr_features = list()
corr_matrix = reduced.corr()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.4:
            colname = corr_matrix.columns[i]
            corr_features.append(str(colname))

list_corr = list()
for item in corr_features:
    list_corr.append(int(item))

df_reduced_corr = reduced.drop(columns = list_corr, axis=0)
df_reduced_corr

In [ ]:
set(list_corr)

In [ ]:
df_reduced_corr = reduced.copy()
for item in tqdm(set(list_corr)):
    df_reduced_corr.drop(columns = str(item), inplace = True)
df_reduced_corr

In [ ]:
feature_n = pd.DataFrame(columns = feature_name)
feature_n

In [ ]:
feature_n.to_csv('Feature.csv', sep=',', encoding='utf-8')

In [ ]:
a = pd.read_csv('./Feature.csv').iloc[:,1:-1]
a

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_reduced_corr, target, test_size=0.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

# **Let's work with complete dataset**

In [ ]:
label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES_INTE.head(40198)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:14]

In [ ]:
DF_LUNG_DISEASES['Finding Labels'][10000:22500].value_counts()

In [ ]:
import cudf
DATA = cudf.read_csv('../input/diseases-dataset-features-50000/features_total.csv', skiprows=10000, nrows=12500).iloc[:,2:-1]
#PANDAS === DATA = (pd.read_csv('../input/diseases-dataset-features-50000/features_total.csv',dtype = np.float16)).iloc[:,1:-1]

In [ ]:
DATA

In [ ]:
import gc
gc.collect()

In [ ]:
arr = np.array(DATA.to_gpu_matrix())

In [ ]:
DATASET = pd.DataFrame(arr)

In [ ]:
del arr
gc.collect()

In [ ]:
DATASET

In [ ]:
targets = DF_LUNG_DISEASES['Finding Labels'][10000:22500]

In [ ]:
targets.value_counts()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(DATASET, targets)
model = SelectFromModel(clf, prefit=True)
new = model.transform(DATASET)
reduced = pd.DataFrame(new)
reduced.to_csv('Reduced.csv', sep=',', encoding='utf-8')
feature_idx = model.get_support()
feature_name = DATASET.columns[feature_idx]
reduced

In [ ]:
feature_name = np.array(feature_name)
feature_name

In [ ]:
feature_n = pd.DataFrame(columns = feature_name)
feature_n.to_csv('Features_rf.csv', sep=',', encoding='utf-8')

In [ ]:
feature_n

In [ ]:
reduced.columns = feature_name
reduced

In [ ]:
reduced.to_csv('Reduced_ok.csv', sep=',', encoding='utf-8')

In [ ]:
reduced = pd.read_csv('../input/features-ok/Reduced_ok.csv',dtype = np.float16).iloc[:,1:-1]

In [ ]:
reduced

In [ ]:
corr_features = list()
corr_matrix = reduced.corr()

for i in tqdm(range(len(corr_matrix.columns))):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.25:
            colname = corr_matrix.columns[i]
            corr_features.append(str(colname))

list_corr = list()
for item in tqdm(corr_features):
    list_corr.append(int(item))

In [ ]:
corr_matrix

In [ ]:
corr_features

In [ ]:
import gc
gc.collect()

In [ ]:
df_reduced_corr

In [ ]:
(set(list_corr))

In [ ]:
haha = reduced.head(10)
haha

In [ ]:
df_reduced_corr = haha.copy()
for item in tqdm(set(list_corr)):
    df_reduced_corr.drop(columns = str(item), inplace = True)
df_reduced_corr

In [ ]:
df_reduced_corr.describe()

In [ ]:
g = pd.DataFrame(columns = df_reduced_corr.columns)

In [ ]:
df_reduced_corr.columns

In [ ]:
df_reduced_corr.to_csv('Reduced_final.csv', sep=',', encoding='utf-8')

In [ ]:
#https://www.kaggle.com/vbookshelf/play-audio-read-the-files-create-a-spectrogram

from pydub import AudioSegment
import IPython

path = '../input/alarm-sound/Alarm_Slow_A1_fesliyanstudios.mp3'

IPython.display.Audio(path, autoplay=True)

# 12. Structuring neural network

In [ ]:
df = pd.read_csv('../input/diseases-dataset-features-50000/features_total.csv', usecols = df_reduced_corr.columns, low_memory = True)

In [ ]:
df

In [ ]:
targets = DF_LUNG_DISEASES['Finding Labels']
targets

In [ ]:
gc.collect()

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df, targets, test_size=0.25, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
fig, ax = plt.subplots(1,1, figsize = (16,16))
confusion_matrix = confusion_matrix(y_C_test, prediction.round())
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)  

In [ ]:
#from tensorflow.keras.applications import VGG16
#from keras.layers import *
#with tpu_strategy.scope(): 
    #img_size_target = 256
    #img_input = Input(shape=(img_size_target, img_size_target, 1))
    #img_conc = Concatenate()([img_input, img_input, img_input])
#    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#    for layer in base_model.layers[:19]:
#        layer.trainable = False  
#
#    base_model.summary()

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df, classes, test_size=0.25, random_state=42)

In [ ]:
#with tpu_strategy.scope(): 
model = models.Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(X_F_train.shape[-1],)))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(y_C_train.shape[-1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_F_train, np.asarray(y_C_train).astype(np.float32), batch_size = 32, epochs = 100, verbose = 1, validation_split=0.20)

# 13. VGG16

In [ ]:
label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
DF_LUNG_DISEASES['diseases'] = DF_LUNG_DISEASES['Finding Labels']
DF_LUNG_DISEASES['diseases'].value_counts()

In [ ]:
print(DF_LUNG_DISEASES['diseases'].value_counts().to_string())

In [ ]:
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)

DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Atelectasis']
DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES_INTE0.head(3215)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE0.index, axis=0)

DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Cardiomegaly']
DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES_INTE1.head(93)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE1.index, axis=0)

DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Consolidation']
DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES_INTE2.head(1310)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE2.index, axis=0)

DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Edema']
DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES_INTE3.head(628)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE3.index, axis=0)

DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Effusion']
DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES_INTE4.head(2955)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE4.index, axis=0)

DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Emphysema']
DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES_INTE5.head(892)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE5.index, axis=0)

DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Fibrosis']
DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES_INTE6.head(727)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE6.index, axis=0)

DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Hernia']
DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES_INTE7.head(110)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE7.index, axis=0)

DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Infiltration']
DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES_INTE8.head(8547)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE8.index, axis=0)

DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Mass']
DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES_INTE9.head(1139)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE9.index, axis=0)

DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES_INTE10.head(59361)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE10.index, axis=0)

DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Nodule']
DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES_INTE11.head(1705)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE11.index, axis=0)

#DF_LUNG_DISEASES_INTE12 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Pneumonia']
#DF_LUNG_DISEASES_INTE12 = DF_LUNG_DISEASES_INTE12.head(0)
#DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE12.index, axis=0)

DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Pneumothorax']
DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES_INTE13.head(1194)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE13.index, axis=0)

DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:9]

In [ ]:
cla =  pd.DataFrame(classes)
cla

In [ ]:
cols = cla.columns.values

In [ ]:
for i,colu in enumerate(cols):
    DF_LUNG_DISEASES[colu] = cla[i]
DF_LUNG_DISEASES

In [ ]:
DF_LUNG_DISEASES['diseases']

In [ ]:
DF_LUNG_DISEASES.iloc[:,14:23]

In [ ]:
DF_LUNG_DISEASES['Finding Labels'].value_counts()

In [ ]:
DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'
#IMAGES_ORIGINAL = list()
#IMAGES_COLLIMATED = list()
IMAGES_COLLIMATED = np.empty((len(NAMES_JPG_1), IMG_HEIGHT, IMG_WIDTH,1), dtype='float16')
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    #IMAGES_ORIGINAL.append(IMAGE1)
    #Change 1 or 2
    PATH2 = DIR_1 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),0)
    IMAGE2 = cv2.resize(IMAGE2, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGES_COLLIMATED[i] = (np.asarray(IMAGE2).reshape((1, SIZE, SIZE, 1)).astype('float16')) / 255.

In [ ]:
IMAGE2.shape

In [ ]:
import gc
del IMAGE2
gc.collect()

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers
Classifiers.models_names()

In [ ]:
with tpu_strategy.scope():
    seresnext50, _ = Classifiers.get('seresnext50')
    base = seresnext50(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')
    
    input_tensor = Input(shape=(SIZE, SIZE,1))
    x = Conv2D(3,(1,1),padding='same')(input_tensor)
    out = base(x) 
    base_model = Model(inputs=input_tensor,outputs=out)
    
    #x= input_tensor(base_model)
    #x= layers.GlobalAveragePooling2D()(layers.Dropout(0.16)(x))
    #x= layers.Dropout(0.3)(x)
    #model = layers.Dense(14, 'sigmoid')(x)
    #model.summary()
    
    model = models.Sequential()
    model.add(base_model)

    model.add(layers.Dropout(0.16))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.3))
    #model.add(layers.Dense(256, activation='relu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(9, activation='softmax'))
    model.summary()

In [ ]:

seresnext50, _ = Classifiers.get('seresnext50')
base = seresnext50(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')

input_tensor = Input(shape=(SIZE, SIZE,1))
x = Conv2D(3,(1,1),padding='same')(input_tensor)
out = base(x) 
base_model = Model(inputs=input_tensor,outputs=out)

#x= input_tensor(base_model)
#x= layers.GlobalAveragePooling2D()(layers.Dropout(0.16)(x))
#x= layers.Dropout(0.3)(x)
#model = layers.Dense(14, 'sigmoid')(x)
#model.summary()

model = models.Sequential()
model.add(base_model)

model.add(layers.Dropout(0.16))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
#model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(9, activation='sigmoid'))
model.summary()

In [ ]:
base_model.summary()

In [ ]:
"""
from tensorflow.keras.applications import VGG16
from keras.layers import *
with tpu_strategy.scope():
    #img_size_target = 256
    #img_input = Input(shape=(img_size_target, img_size_target, 1))
    #img_conc = Concatenate()([img_input, img_input, img_input])
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
    
    for layer in base_model.layers[:19]:
        layer.trainable = False
        
    input_tensor = Input(shape=(SIZE, SIZE,1))
    x = Conv2D(3,(3,3),padding='same')(input_tensor)
    out = base_model (x) 
    base_model = Model(inputs=input_tensor,outputs=out)
    base_model.summary()

In [ ]:
"""
from tensorflow.keras.applications import ResNet50
from keras.layers import *
with tpu_strategy.scope():
    resnet = ResNet50(weights='imagenet',include_top= False) 

    input_tensor = Input(shape=(SIZE, SIZE,1))
    x = Conv2D(3,(3,3),padding='same')(input_tensor)
    out = resnet (x) 
    base_model = Model(inputs=input_tensor,outputs=out)
    base_model.summary()

In [ ]:
"""
with tpu_strategy.scope():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(14, activation='sigmoid'))
    model.summary()

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True)])

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(IMAGES_COLLIMATED, DF_LUNG_DISEASES,train_size = 0.80,test_size=0.20, random_state=42)

In [ ]:
y_C_test

In [ ]:
y_C_test.iloc[:,14:23]

In [ ]:
y_C_train.to_csv('df_train_val.csv')

In [ ]:
y_C_test

In [ ]:
len(X_F_train)

In [ ]:
np.asarray(y_C_train.iloc[:,14:23])[0].astype('int8').mean()

In [ ]:
import gc
del IMAGES_COLLIMATED
gc.collect()

In [ ]:
#Callbacks: https://www.kaggle.com/akhileshdkapse/nih-x-ray-multilabel-classification-tpu-guide
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
name= 'NIH_seresnext50_model_256_no_col.h5'
rlr = ReduceLROnPlateau(monitor = 'val_auc_1', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_auc_1',verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_auc_1', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(IMAGES_COLLIMATED, np.asarray(DF_LUNG_DISEASES.iloc[:,14:23]).astype('float16'), batch_size = 8 * tpu_strategy.num_replicas_in_sync, epochs = 100, verbose = 1, validation_split=0.2,shuffle=True,callbacks=[rlr,es,ckp])

In [ ]:
#Callbacks: https://www.kaggle.com/akhileshdkapse/nih-x-ray-multilabel-classification-tpu-guide
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
name= 'NIH_seresnext50_model_256_no_col.h5'
rlr = ReduceLROnPlateau(monitor = 'val_auc_5', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_auc_5',verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_auc_5', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(X_F_train, np.asarray(y_C_train.iloc[:,14:23]).astype('int8'), batch_size =  19 * tpu_strategy.num_replicas_in_sync, epochs = 100, verbose = 1, validation_data = (X_F_test,np.asarray(y_C_test.iloc[:,14:23]).astype('int8')),shuffle=True,callbacks=[rlr,es,ckp])

In [ ]:
DF_LUNG_DISEASES["Finding Labels"]

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(np.argmax(np.asarray(y_C_train.iloc[:,14:23]), axis=1)), np.argmax(np.asarray(y_C_train.iloc[:,14:23].values), axis=1))

In [ ]:
class_weights

In [ ]:
class_weights = dict(enumerate(class_weights))

In [ ]:
#Callbacks: https://www.kaggle.com/akhileshdkapse/nih-x-ray-multilabel-classification-tpu-guide
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
name= 'NIH_seresnext50_model_256_no_col.h5'
rlr = ReduceLROnPlateau(monitor = 'val_auc', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_auc',verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_auc', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(X_F_train, np.asarray(y_C_train.iloc[:,14:23]).astype('float16'), batch_size = 32, epochs = 100, verbose = 1, validation_data = (X_F_test,np.asarray(y_C_test.iloc[:,14:23]).astype('float16')),shuffle=True,class_weight=class_weights,callbacks=[rlr,es,ckp])

In [ ]:
#validation_data = (X_F_test,np.asarray(y_C_test.iloc[:,14:23]).astype('float16'))

In [ ]:
model_= tf.keras.models.load_model('./NIH_seresnext50_model_256_no_col.h5')
#pred= model_.predict(X_F_test, verbose=1)

In [ ]:
model_.evaluate(X_F_test,np.asarray(y_C_test.iloc[:,14:28]).astype('float16')) 

In [ ]:
label_names

In [ ]:
DF_LUNG_DISEASES['diseases'].value_counts()

In [ ]:
DF_LUNG_DISEASES['Finding Labels'].value_counts()

In [ ]:
y_C_test.iloc[:,14:28].value_counts()

In [ ]:
y_C_test['diseases'].value_counts()

In [ ]:
del X_F_train
gc.collect()

In [ ]:
new_label_names = ['Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','No Finding','Nodule','Pneumonia','Pneumothorax']

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

predictions = model_.predict(X_F_test,verbose =1) 
test_Y = np.asarray(y_C_test.iloc[:,14:23]).astype('float16')

fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for i, name in enumerate(new_label_names): 
    #for line in range(0,len(predictions)):
    #    predictions[line][predictions[line]==max(predictions[line])] = 1
    #    predictions[line][predictions[line]<max(predictions[line])] = 0
    #class_test = [np.argmax(t) for t in test_Y]
    #pred2 = [np.argmax(t) for t in predictions]
    fpr, tpr, thresholds = roc_curve(test_Y[:,i].astype(int), predictions[:,i])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (name, auc(fpr, tpr)))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('quick_trained_model_128_col.png')


In [ ]:
np.argmax(predictions.round(),axis = 1)

In [ ]:
test_Y

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

confusion_matrix = confusion_matrix(np.argmax(test_Y,axis=1), np.argmax(predictions.round(),axis = 1))
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
with tpu_strategy.scope():
    seresnext50, _ = Classifiers.get('seresnext50')
    base = seresnext50(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')
    
    input_tensor = Input(shape=(SIZE, SIZE,1))
    x = Conv2D(3,(3,3),padding='same')(input_tensor)
    out = base(x) 
    base_model = Model(inputs=input_tensor,outputs=out)
    
    #x= input_tensor(base_model)
    #x= layers.GlobalAveragePooling2D()(layers.Dropout(0.16)(x))
    #x= layers.Dropout(0.3)(x)
    #model = layers.Dense(14, 'sigmoid')(x)
    #model.summary()
    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Dropout(0.16))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.3))
    #model.add(layers.Dense(256, activation='relu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(14, activation='sigmoid'))
    model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(multi_label=True)])
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(IMAGES_COLLIMATED, DF_LUNG_DISEASES,train_size = 0.80,test_size=0.20, random_state=42)

#Callbacks: https://www.kaggle.com/akhileshdkapse/nih-x-ray-multilabel-classification-tpu-guide
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
name= 'NIH_seresnext50_model_128_no_col.h5'
rlr = ReduceLROnPlateau(monitor = 'val_auc', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_auc',verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_auc', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(X_F_train, np.asarray(y_C_train.iloc[:,14:28]).astype('float16'), batch_size = 8 * tpu_strategy.num_replicas_in_sync, epochs = 100, verbose = 1, validation_data = (X_F_test,np.asarray(y_C_test.iloc[:,14:28]).astype('float16')),shuffle=True,callbacks=[rlr,es,ckp])

new_label_names = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema','Fibrosis','Hernia','Infiltration','Mass','No Finding','Nodule','Pneumonia','Pneumothorax']
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

predictions = model_.predict(X_F_test,verbose =1) 
test_Y = np.asarray(y_C_test.iloc[:,14:28]).astype('float16')

fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for i, name in enumerate(new_label_names): 
    #for line in range(0,len(predictions)):
    #    predictions[line][predictions[line]==max(predictions[line])] = 1
    #    predictions[line][predictions[line]<max(predictions[line])] = 0
    #class_test = [np.argmax(t) for t in test_Y]
    #pred2 = [np.argmax(t) for t in predictions]
    fpr, tpr, thresholds = roc_curve(test_Y[:,i].astype(int), predictions[:,i])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (name, auc(fpr, tpr)))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('quick_trained_model_128_no_col.png')

In [ ]:
roc_curve(test_Y[:,i].astype(int), predictions[:,i].astype(int))

In [ ]:
test_Y[:,i].astype(int)

In [ ]:
predictions[0][predictions[0]<max(predictions[0])] = 0
predictions

In [ ]:
roc_curve(test_Y[:,i].astype(int), predictions[:,i].astype(int))

In [ ]:
fpr, tpr, thresholds = roc_curve(np.squeeze(test_Y[:,[i]].astype(int)), np.squeeze(predictions[:,[i]].astype(int)))

In [ ]:
print('oi')

In [ ]:
predictions

In [ ]:
np.squeeze(test_Y[:,0].astype(int))

In [ ]:
np.squeeze(predictions[:,0].astype(int))

In [ ]:
pred = (pred > 0.5)
class_test = [np.argmax(t) for t in X_F_test]
pred2 = [np.argmax(t) for t in pred]


# 13. Structuring gridsearch and crossvalidation applied to classification 

In [ ]:
SIZE = 256
CHANNELS = 1

IMG_HEIGHT, IMG_WIDTH = 256,256
SEED=1
NUM_SAMPLES = 4000
BATCH_SIZE = 1
EPOCHS = 100

DF_LUNG_DISEASES=pd.read_csv("../input/data/Data_Entry_2017.csv")
LUNG_DISEASES_DIR = os.path.join('../input/data')

label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
DF_LUNG_DISEASES['diseases'] = DF_LUNG_DISEASES['Finding Labels']
DF_LUNG_DISEASES['diseases'].value_counts()
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)

DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Atelectasis']
DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES_INTE0.head(3215)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE0.index, axis=0)

DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Cardiomegaly']
DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES_INTE1.head(93)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE1.index, axis=0)

DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Consolidation']
DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES_INTE2.head(1310)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE2.index, axis=0)

DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Edema']
DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES_INTE3.head(628)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE3.index, axis=0)

DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Effusion']
DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES_INTE4.head(2955)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE4.index, axis=0)

DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Emphysema']
DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES_INTE5.head(892)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE5.index, axis=0)

DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Fibrosis']
DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES_INTE6.head(727)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE6.index, axis=0)

DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Hernia']
DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES_INTE7.head(110)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE7.index, axis=0)

DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Infiltration']
DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES_INTE8.head(8547)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE8.index, axis=0)

DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Mass']
DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES_INTE9.head(1139)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE9.index, axis=0)

DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES_INTE10.head(59361)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE10.index, axis=0)

DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Nodule']
DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES_INTE11.head(1705)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE11.index, axis=0)

DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Pneumothorax']
DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES_INTE13.head(1194)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE13.index, axis=0)

DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:9]
cla =  pd.DataFrame(classes)
cols = cla.columns.values

for i,colu in enumerate(cols):
    DF_LUNG_DISEASES[colu] = cla[i]
DF_LUNG_DISEASES

DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'

dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model_autoencoder = load_model('../input/autoencoder-supression-300-epochs/Autoencoder_300_Xnet_supression_LearningRateScheduler (1).h5', custom_objects=dependencies)
loaded_model_autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

#IMAGES_ORIGINAL = list()
#IMAGES_COLLIMATED = list()
IMAGES_COLLIMATED = np.empty((len(NAMES_JPG_1), IMG_HEIGHT, IMG_WIDTH,1), dtype='float16')
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    #IMAGES_ORIGINAL.append(IMAGE1)
    #Change 1, 2 or 3
    PATH2 = DIR_1 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),0)
    IMAGE2 = cv2.equalizeHist(IMAGE2)
    IMAGE2 = (np.asarray(IMAGE2).reshape(1,SIZE, SIZE, CHANNELS).astype('float16')) / 255
    IMAGE2 = loaded_model_autoencoder.predict(IMAGE2.astype(np.float16))
    IMAGE2 = np.squeeze(IMAGE2.astype(np.float32))
    IMAGE2 = cv2.resize(IMAGE2, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    #IMAGE2 = cv2.equalizeHist(IMAGE2)
    IMAGES_COLLIMATED[i] = (np.asarray(IMAGE2).reshape((1, SIZE, SIZE, 1)).astype('float16'))
    #/ 255.
    #number = 0
    #fig, ax = plt.subplots(1,1, figsize = (8,4))
    #plt.imshow(IMAGE2, cmap='gray')
    #plt.show()
    
os.makedirs('./DISEASES_supre/')
for i in tqdm(range(0,len(NAMES_JPG_1))):
    im = np.squeeze(IMAGES_COLLIMATED[i].astype(np.float32))
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    plt.imsave('./DISEASES_supre/'+ str(NAMES_JPG_1.iloc[i]),im, cmap = 'gray')

#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers

#X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(IMAGES_COLLIMATED, DF_LUNG_DISEASES,train_size = 0.80,test_size=0.20, random_state=42)

In [ ]:
number = 0
fig, ax = plt.subplots(1,1, figsize = (8,4))
plt.imshow(IMAGE2, cmap='gray')
plt.show()

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./DISEASES_supre.zip', 'w')
 
for folder, subfolders, files in os.walk('./DISEASES_supre'):
 
    for file in files:
        if file.endswith('.jpg'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './DISEASES'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

# New beginning

In [ ]:
with zipfile.ZipFile("../input/supre2/DISEASES_supre.zip","r") as zip_ref:
    zip_ref.extractall("DISEASES_supre")

In [ ]:
SIZE = 256
CHANNELS = 1

IMG_HEIGHT, IMG_WIDTH = 256,256
SEED=1
NUM_SAMPLES = 4000
BATCH_SIZE = 1
EPOCHS = 100

DF_LUNG_DISEASES=pd.read_csv("../input/data/Data_Entry_2017.csv")
LUNG_DISEASES_DIR = os.path.join('../input/data')

label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
DF_LUNG_DISEASES['diseases'] = DF_LUNG_DISEASES['Finding Labels']
DF_LUNG_DISEASES['diseases'].value_counts()
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)

DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Atelectasis']
DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES_INTE0.head(3215)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE0.index, axis=0)

DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Cardiomegaly']
DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES_INTE1.head(93)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE1.index, axis=0)

DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Consolidation']
DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES_INTE2.head(1310)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE2.index, axis=0)

DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Edema']
DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES_INTE3.head(628)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE3.index, axis=0)

DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Effusion']
DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES_INTE4.head(2955)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE4.index, axis=0)

DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Emphysema']
DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES_INTE5.head(892)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE5.index, axis=0)

DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Fibrosis']
DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES_INTE6.head(727)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE6.index, axis=0)

DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Hernia']
DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES_INTE7.head(110)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE7.index, axis=0)

DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Infiltration']
DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES_INTE8.head(8547)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE8.index, axis=0)

DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Mass']
DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES_INTE9.head(1139)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE9.index, axis=0)

DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES_INTE10.head(59361)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE10.index, axis=0)

DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Nodule']
DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES_INTE11.head(1705)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE11.index, axis=0)

DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Pneumothorax']
DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES_INTE13.head(1194)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE13.index, axis=0)

DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:9]
cla =  pd.DataFrame(classes)
cols = cla.columns.values

for i,colu in enumerate(cols):
    DF_LUNG_DISEASES[colu] = cla[i]
DF_LUNG_DISEASES

DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'
DIR_3 = './DISEASES_supre/DISEASES_supre/'
#IMAGES_ORIGINAL = list()
#IMAGES_COLLIMATED = list()
IMAGES_COLLIMATED = np.empty((len(NAMES_JPG_1), IMG_HEIGHT, IMG_WIDTH,1), dtype='float16')
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    #IMAGES_ORIGINAL.append(IMAGE1)
    #Change 1 or 2
    PATH2 = DIR_3 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),0)
    #IMAGE2 = np.squeeze(IMAGE2.astype(np.float32))
    IMAGE2 = cv2.resize(IMAGE2, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGES_COLLIMATED[i] = (np.asarray(IMAGE2).reshape((1, SIZE, SIZE, 1)).astype('float16')) / 255.

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers

#X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(IMAGES_COLLIMATED, DF_LUNG_DISEASES,train_size = 0.80,test_size=0.20, random_state=42)

In [ ]:
DF_LUNG_DISEASES=pd.read_csv("../input/data/Data_Entry_2017.csv")
LUNG_DISEASES_DIR = os.path.join('../input/data')

label_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule",'No Finding',"Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]
DF_LUNG_DISEASES['diseases'] = DF_LUNG_DISEASES['Finding Labels']
DF_LUNG_DISEASES['diseases'].value_counts()
disease_of_interest = list()
for i in tqdm(range(0,DF_LUNG_DISEASES["Finding Labels"].count())):
    if DF_LUNG_DISEASES["Finding Labels"].iloc[i] in label_names:
        class_ = 1
    else:
        class_ = 0 
    disease_of_interest.append(class_)
    
DF_LUNG_DISEASES['disease_of_interest'] = disease_of_interest
DF_LUNG_DISEASES_INTE = DF_LUNG_DISEASES[DF_LUNG_DISEASES['disease_of_interest'] == 0 ]
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE.index, axis=0)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)

DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Atelectasis']
DF_LUNG_DISEASES_INTE0 = DF_LUNG_DISEASES_INTE0.head(3215)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE0.index, axis=0)

DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Cardiomegaly']
DF_LUNG_DISEASES_INTE1 = DF_LUNG_DISEASES_INTE1.head(93)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE1.index, axis=0)

DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Consolidation']
DF_LUNG_DISEASES_INTE2 = DF_LUNG_DISEASES_INTE2.head(1310)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE2.index, axis=0)

DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Edema']
DF_LUNG_DISEASES_INTE3 = DF_LUNG_DISEASES_INTE3.head(628)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE3.index, axis=0)

DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Effusion']
DF_LUNG_DISEASES_INTE4 = DF_LUNG_DISEASES_INTE4.head(2955)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE4.index, axis=0)

DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Emphysema']
DF_LUNG_DISEASES_INTE5 = DF_LUNG_DISEASES_INTE5.head(892)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE5.index, axis=0)

DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Fibrosis']
DF_LUNG_DISEASES_INTE6 = DF_LUNG_DISEASES_INTE6.head(727)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE6.index, axis=0)

DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Hernia']
DF_LUNG_DISEASES_INTE7 = DF_LUNG_DISEASES_INTE7.head(110)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE7.index, axis=0)

DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Infiltration']
DF_LUNG_DISEASES_INTE8 = DF_LUNG_DISEASES_INTE8.head(8547)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE8.index, axis=0)

DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Mass']
DF_LUNG_DISEASES_INTE9 = DF_LUNG_DISEASES_INTE9.head(1139)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE9.index, axis=0)

DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'No Finding']
DF_LUNG_DISEASES_INTE10 = DF_LUNG_DISEASES_INTE10.head(59361)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE10.index, axis=0)

DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Nodule']
DF_LUNG_DISEASES_INTE11 = DF_LUNG_DISEASES_INTE11.head(1705)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE11.index, axis=0)

DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES[DF_LUNG_DISEASES['Finding Labels'] == 'Pneumothorax']
DF_LUNG_DISEASES_INTE13 = DF_LUNG_DISEASES_INTE13.head(1194)
DF_LUNG_DISEASES = DF_LUNG_DISEASES.drop(DF_LUNG_DISEASES_INTE13.index, axis=0)

DF_LUNG_DISEASES = DF_LUNG_DISEASES.reset_index(drop=True)
NAMES_JPG_1 = DF_LUNG_DISEASES['Image Index'].str.replace('.png', '.jpg')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelencorder_targets = LabelEncoder()
DF_LUNG_DISEASES['Finding Labels'] = labelencorder_targets.fit_transform(DF_LUNG_DISEASES['Finding Labels'])

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(),[1])],remainder='passthrough')
predictors = onehotencorder.fit_transform(DF_LUNG_DISEASES)
classes = predictors[:,0:9]
cla =  pd.DataFrame(classes)
cols = cla.columns.values

for i,colu in enumerate(cols):
    DF_LUNG_DISEASES[colu] = cla[i]
DF_LUNG_DISEASES

DIR_1 = './images/DISEASES/images/'
DIR_2 = './attention_images/Diseases/attention/'
DIR_3 = './DISEASES_supre/DISEASES_supre/'
#IMAGES_ORIGINAL = list()
#IMAGES_COLLIMATED = list()
IMAGES_COLLIMATED = np.empty((len(NAMES_JPG_1), IMG_HEIGHT, IMG_WIDTH,1), dtype='float16')
for i in tqdm(range(0,len(NAMES_JPG_1))):
    #PATH1 = DIR_1 + NAMES_JPG_1.iloc[i]
    #IMAGE1 = cv2.imread(os.path.join(PATH1),cv2.IMREAD_GRAYSCALE)
    #IMAGES_ORIGINAL.append(IMAGE1)
    #Change 1 or 2
    PATH2 = DIR_1 + NAMES_JPG_1.iloc[i]
    IMAGE2 = cv2.imread(os.path.join(PATH2),0)
    #IMAGE2 = np.squeeze(IMAGE2.astype(np.float32))
    IMAGE2 = cv2.resize(IMAGE2, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGES_COLLIMATED[i] = (np.asarray(IMAGE2).reshape((1, SIZE, SIZE, 1)).astype('float16')) / 255.

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers

#X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(IMAGES_COLLIMATED, DF_LUNG_DISEASES,train_size = 0.80,test_size=0.20, random_state=42)

In [ ]:
number = 0
fig, ax = plt.subplots(1,1, figsize = (8,4))
plt.imshow(IMAGE2, cmap='gray')
plt.show()

# Normal

In [ ]:
with tpu_strategy.scope():
    seresnext50, _ = Classifiers.get('seresnext50')
    base = seresnext50(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')
    
    input_tensor = Input(shape=(SIZE, SIZE,1))
    x = Conv2D(3,(1,1),padding='same')(input_tensor)
    out = base(x) 
    base_model = Model(inputs=input_tensor,outputs=out)
    
    model = models.Sequential()
    model.add(base_model)

    model.add(layers.Dropout(0.16))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.3))
    #model.add(layers.Dense(256, activation='relu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(9, activation='softmax'))
    model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.AUC(multi_label=True)])


In [ ]:
#Callbacks: https://www.kaggle.com/akhileshdkapse/nih-x-ray-multilabel-classification-tpu-guide
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
name= 'NIH_seresnext50_model_256_no_col.h5'
rlr = ReduceLROnPlateau(monitor = 'val_auc_2', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_auc_2',verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_auc_2', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(X_F_train, np.asarray(y_C_train.iloc[:,14:23]).astype('float16'), batch_size =  19 * tpu_strategy.num_replicas_in_sync, epochs = 100, verbose = 1, validation_data = (X_F_test,np.asarray(y_C_test.iloc[:,14:23]).astype('float16')),shuffle=True,callbacks=[rlr,es,ckp])

# Cross validation

In [ ]:
ppr = [0] * 10
for i in range(10):
    ppr[i] = [0] * 9
ppr

In [ ]:
#fpr[j][i] 
ppr[0][6] = 2
ppr

In [ ]:
import numpy as np

ppr_ = np.array(ppr)

In [ ]:
ppr_[:,6]

In [ ]:
ppr_[:,6].mean()

In [ ]:
ppr_[0][6]

In [ ]:
pip install np_utils

In [ ]:
#from tensorflow.keras.utils import np_utils

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
resultados = []
j = 0

fpr = [0] * 10
for i in range(10):
    fpr[i] = [0] * 9
    
tpr = [0] * 10
for i in range(10):
    tpr[i] = [0] * 9

def Average(lst):
    return sum(lst) / len(lst)

for train_index, test_index in kfold.split(IMAGES_COLLIMATED,np.zeros(shape = (IMAGES_COLLIMATED.shape[0],1))):
    
    with tpu_strategy.scope():
        seresnext50, _ = Classifiers.get('seresnext50')
        base = seresnext50(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')
        input_tensor = Input(shape=(SIZE, SIZE,1))
        x = Conv2D(3,(1,1),padding='same')(input_tensor)
        out = base(x) 
        base_model = Model(inputs=input_tensor,outputs=out)
        model = models.Sequential()
        model.add(base_model)
        model.add(layers.Dropout(0.16))
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dropout(0.3))
        model.add(layers.Dense(9, activation='softmax'))
        model.summary()

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True)])
    
    if j == 0:
        k = 'val_auc'
    else:
        k = 'val_auc_'+str(j)
    
    name= 'NIH_seresnext50_model_256_no_col.h5'
    rlr = ReduceLROnPlateau(monitor = k, factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
    ckp = ModelCheckpoint(name,monitor = k,verbose = 1, save_best_only = True, mode = 'max')    
    es = EarlyStopping(monitor = k, min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
    history = model.fit(IMAGES_COLLIMATED[train_index], np.asarray(DF_LUNG_DISEASES.iloc[:,14:23])[train_index].astype('int8'), batch_size = 128, epochs = 100, validation_data = (IMAGES_COLLIMATED[test_index], np.asarray(DF_LUNG_DISEASES.iloc[:,14:23])[test_index].astype('int8')),callbacks=[rlr,es,ckp])
    
    model_= tf.keras.models.load_model('./NIH_seresnext50_model_256_no_col.h5')
    precision = model_.evaluate(IMAGES_COLLIMATED[test_index], np.asarray(DF_LUNG_DISEASES.iloc[:,14:23])[test_index].astype('int8'), batch_size = 128)
    new_label_names = ['Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','No Finding','Nodule','Pneumonia','Pneumothorax']
    predictions = model_.predict(IMAGES_COLLIMATED[test_index]) 

    for i, name in enumerate(new_label_names): 
        fpr[j][i], tpr[j][i], thresholds = roc_curve(np.asarray(DF_LUNG_DISEASES.iloc[:,14:23])[test_index].astype('int8')[:,i].astype(int), predictions[:,i])

    resultados.append(precision[1])
    j += 1

In [ ]:
resultados

In [ ]:
#https://www.kaggle.com/vbookshelf/play-audio-read-the-files-create-a-spectrogram

from pydub import AudioSegment
import IPython

path = '../input/alarm-sound/Alarm_Slow_A1_fesliyanstudios.mp3'

IPython.display.Audio(path, autoplay=True)

# Testes

In [ ]:
fpr_[3,0].shape

In [ ]:
fpr_ = np.array(fpr)
tpr_ = np.array(tpr)

fpr_m = [0]*9
tpr_m = [0]*9

for j in range(9):
    fpr_m[j] = fpr_[:,j].mean()
    tpr_m[j] = tpr_[:,j].mean()
    
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for j in range(1):
    for i, name in enumerate(new_label_names):
        c_ax.plot(fpr_m[j][i], tpr_m[j][i], label = '%s (AUC:%0.2f)'  % (name, auc(fpr_m[j][i], tpr_m[j][i])))
    c_ax.legend()
    c_ax.set_xlabel('False Positive Rate')
    c_ax.set_ylabel('True Positive Rate')
    fig.savefig('quick_trained_model_256_no_col_1.png')

In [ ]:
resultados

# Grid (memory error)

In [ ]:
def network(type_network):    
    K.clear_session()
    with tpu_strategy.scope():
        net, _ = Classifiers.get(type_network)
        base = net(input_shape=(SIZE, SIZE, 3), include_top=False, weights='imagenet')

        input_tensor = Input(shape=(SIZE, SIZE,1))
        x = Conv2D(3,(1,1),padding='same')(input_tensor)
        out = base(x) 
        base_model = Model(inputs=input_tensor,outputs=out)

        model = models.Sequential()
        model.add(base_model)

        model.add(layers.Dropout(0.16))
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dropout(0.3))
        #model.add(layers.Dense(256, activation='relu'))
        #model.add(layers.Dropout(0.5))
        model.add(layers.Dense(9, activation='softmax'))
        model.summary()

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.AUC(multi_label=True)])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

classifier = KerasClassifier(build_fn = network)
parameters = {'batch_size': [19 * tpu_strategy.num_replicas_in_sync], 'epochs': [10],'type_network':['resnet18','resnet34']}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = tf.keras.metrics.AUC(multi_label=True), cv = 2)
grid_search = grid_search.fit(IMAGES_COLLIMATED, np.asarray(DF_LUNG_DISEASES.iloc[:,14:23]).astype('float16'))

In [ ]:
pd.DataFrame(grid_search.cv_results_)

PATH2PATH2os.path.join(PATH2)

In [ ]:
import shutil

shutil.rmtree('./Diseases')